In [52]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.path as mplPath

In [109]:
df = pd.read_csv("train_contest.csv")
df.head(4)

,Id,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,TARGET
0,57094,1.0,3,0,2016-05-19 18:06:27,A FABULOUS 3BR IN MIDTOWN WEST! PERFECT APAR...,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,"['Laundry In Unit', 'No Fee', 'Elevator']",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10,medium
1,33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,Renovated Kitchen and Bathroom!,55 River Drive South,"['Dogs Allowed', 'Cats Allowed', 'No Fee']",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South,medium
2,60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,RARE AND BEST DEAL ON THE MARKET!!!! PERFECT S...,W 77 Street,"['Elevator', 'Hardwood Floors']",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street,low
3,53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,Newly renovated flex 2 apartment offers the ne...,John Street,"['Swimming Pool', 'Doorman', 'Elevator', 'Fitn...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street,low


In [7]:
df.drop('Id', axis=1, inplace=True)

In [8]:
df.isna().sum()

bathrooms             0
bedrooms              0
building_id           0
created               0
description        1037
display_address      88
features              0
latitude              0
listing_id            0
longitude             0
manager_id            0
photos                0
price                 0
street_address        4
TARGET                0
dtype: int64

In [9]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [9]:
df['bathrooms'].unique()

array([1. , 2. , 2.5, 3. , 1.5, 0. , 4. , 3.5, 4.5, 5. , 5.5, 6. ])

In [11]:
df['bedrooms'].unique()

array([3, 1, 0, 2, 4, 5, 6, 8], dtype=int64)

In [12]:
df['building_id'].nunique()

6378

In [13]:
sum(df['building_id'] == '0') # видимо '0' - аналог NaN

5713

In [10]:
df['month'] = pd.to_datetime(df['created']).apply(lambda x : x.month)
df['day'] = pd.to_datetime(df['created']).apply(lambda x : x.day)
df.drop('created', axis=1, inplace=True)

In [11]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import wordnet, pos_tag
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Никита\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Никита\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
def get_wordnet_pos(treebank_tag):
    my_switch = {
        'J': wordnet.wordnet.ADJ,
        'V': wordnet.wordnet.VERB,
        'N': wordnet.wordnet.NOUN,
        'R': wordnet.wordnet.ADV,
    }
    for key, item in my_switch.items():
        if treebank_tag.startswith(key):
            return item
    return wordnet.wordnet.NOUN

In [13]:
def my_lemmatizer(sent):
    lemmatizer = WordNetLemmatizer()
    
    tokenized_sent = sent.split()
    pos_tagged = [(word, get_wordnet_pos(tag))
                 for word, tag in pos_tag(tokenized_sent)]
    return [lemmatizer.lemmatize(word, tag) for word, tag in pos_tagged]

In [14]:
import re, string

def preprocess_text(text): 
    text = text.lower()
    stop_words = set(['those', 'on', 'own', '’ve', 'yourselves', 
                  'around', 'between', 'four', 'been', 'alone',
                  'off', 'am', 'then', 'other', 'can', 'regarding', 
                  'hereafter', 'front', 'too', 'used', 'wherein', '‘ll',
                  'doing', 'everything', 'up', 'onto', 'never', 'either', 
                  'how', 'before', 'anyway', 'since', 'through', 'amount', 
                  'now', 'he', 'was', 'have', 'into', 'because', 
                  'therefore', 'they', 'n’t', 'even', 'whom', 'it', 'see',
                  'somewhere', 'thereupon', 'nothing', 'whereas', 'much', 
                  'whenever', 'seem', 'until', 'whereby', 'at', 'also', 
                  'some', 'last', 'than', 'get', 'already', 'our', 'once', 
                  'will', 'noone', "'m", 'that', 'what', 'thus', 
                  'myself', 'out', 'next', 'whatever', 'although', 'though',
                  'which', 'would', 'therein', 'nor', 'somehow', 'whereupon',
                  'besides', 'whoever', 'ourselves', 'few', 'did', 'without',
                  'third', 'anything', 'twelve', 'against', 'while', 'twenty',
                  'if', 'however', 'herself', 'when', 'may', 'ours', 'six', 
                  'done', 'seems', 'else', 'call', 'perhaps', 'had', 'nevertheless',
                  'where', 'otherwise', 'still', 'within', 'its', 'for', 'together',
                  'elsewhere', 'throughout', 'of', 'others', 'show', '’s', 'anywhere',
                  'anyhow', 'as', 'are', 'the', 'hence', 'something', 'hereby', 'nowhere',
                  'latterly', 'say', 'does', 'neither', 'his', 'go', 'forty', 'put', 
                  'their', 'by', 'namely', 'could', 'five', 'unless', 'itself', 'is',
                  'nine', 'whereafter', 'down', 'bottom', 'thereby', 'such', 'both',
                  'she', 'become', 'whole', 'who', 'yourself', 'every', 'thru', 'except', 
                  'very', 'several', 'among', 'being', 'be', 'mine', 'further', 'n‘t', 
                  'here', 'during', 'why', 'with', 'just', "'s", 'becomes', '’ll', 'about',
                  'a', 'using', 'seeming', "'d", "'ll", "'re", 'due', 'wherever', 
                  'beforehand', 'fifty', 'becoming', 'might', 'amongst', 'my', 'empty',
                  'thence', 'thereafter', 'almost', 'least', 'someone', 'often', 'from',
                  'keep', 'him', 'or', '‘m', 'top', 'her', 'nobody', 'sometime', 'across',
                  '‘s', '’re', 'hundred', 'only', 'via', 'name', 'eight', 'three', 'back',
                  'to', 'all', 'became', 'move', 'me', 'we', 'formerly', 'so', 'i', 
                  'whence', 'under', 'always', 'himself', 'in', 'herein', 'more', 'after',
                  'themselves', 'you', 'above', 'sixty', 'them', 'your', 'made', 'indeed',
                  'most', 'everywhere', 'fifteen', 'but', 'must', 'along', 'beside', 'hers',
                  'side', 'former', 'anyone', 'full', 'has', 'yours', 'whose', 'behind', 
                  'please', 'ten', 'seemed', 'sometimes', 'should', 'over', 'take', 'each', 
                  'same', 'rather', 'really', 'latter', 'and', 'ca', 'hereupon', 'part', 
                  'per', 'eleven', 'ever', '‘re', 'enough', "n't", 'again', '‘d', 'us', 
                  'yet', 'moreover', 'mostly', 'one', 'meanwhile', 'whither', 'there', 
                  'toward', '’m', "'ve", '’d', 'give', 'do', 'an', 'quite', 'these', 
                  'everyone', 'towards', 'this', 'cannot', 'afterwards', 'beyond', 
                  'make', 'were', 'whether', 'well', 'another', 'below', 'first', 
                  'upon', 'any', 'none', 'many', 'serious', 'various', 're', 'two', 'less', 
                  '‘ve', 'to', 'be', 'a', 'the', 'in', 'room', 'apartment', 'feature', 'br', 'bedroom',
                  'bathroom', 'bath', 'new york', 'p', 'a website redacted', 'contact', 'building', 
                  'floor', 'kitchen', 'email', 'kagglemanagerrenthopcom'])
    text = re.sub(r'(cell)|(contact).*$', ' ', text)
    text = re.sub(r'(<[^>]*>?)|(\([^\)]*\(?)', ' ', text)
    text = re.sub(r'new york', ' ', text)
    text = re.sub(r'[^\w\s]+|[\d]+', ' ', text)
    regex = re.compile('[%s]+' % re.escape(string.punctuation))
    text = regex.sub(' ', text)
    text = re.sub(r'\s+', ' ', text)
    lemmas = my_lemmatizer(text)
    text = ' '.join([lemma for lemma in lemmas if lemma not in stop_words])
    
    return text

In [15]:
df['description'].fillna('', inplace=True)
df['description'] = df['description'].apply(lambda x : preprocess_text(x))
df['description']

0        fabulous midtown west perfect update hardwood ...
1                                                 renovate
2        rare best deal market perfect studio prime upp...
3        newly renovate flex offer new standard luxury ...
4        low fee not come like unit laundry low fee uni...
                               ...                        
34541    newly renovate locate th avenue n q train stat...
34542    newly renovate bleached plank hardwood recess ...
34543    luxury postwar hi rise prime pantry select uni...
34544    coopercooper com list id west th street apt g ...
34545    renovate beautiful hardwood gorgeous natural s...
Name: description, Length: 34546, dtype: object

In [16]:
cv = CountVectorizer()
count = cv.fit_transform(df['description'])
count_values = count.toarray().sum(axis=0)
vocab = cv.vocabulary_
df_count = pd.DataFrame(sorted([(count_values[i], k) for k,i in vocab.items()], reverse=True)
            ).rename(columns={0: 'count', 1:'grams'})
df_count

,count,grams
0,14365,space
1,13684,appliance
2,13653,closet
3,12678,large
4,12618,hardwood
...,...,...
22165,1,abbey
22166,1,abatement
22167,1,abalcony
22168,1,aaronbond


In [17]:
words_to_delete = df_count.loc[df_count['count'] <= 50, 'grams'].values

In [18]:
def delete_words(text):
    splitted_text = text.split()
    
    return ' '.join([token for token in splitted_text if token not in words_to_delete])

In [19]:
df['description'] = df['description'].apply(lambda x : delete_words(x))

In [20]:
cv = CountVectorizer()
count = cv.fit_transform(df['description'])
count_values = count.toarray().sum(axis=0)
vocab = cv.vocabulary_
df_count = pd.DataFrame(sorted([(count_values[i], k) for k,i in vocab.items()], reverse=True)
            ).rename(columns={0: 'count', 1:'grams'})
df_count.shape

(1950, 2)

In [24]:
df_count[df_count['count'] > 50]

,count,grams
0,14365,space
1,13684,appliance
2,13653,closet
3,12678,large
4,12618,hardwood
...,...,...
1945,51,core
1946,51,circular
1947,51,captivate
1948,51,bill


In [22]:
df['photo'] = df.photos.apply(lambda x : 1 if len(x) > 2 else 0)
df.drop('photos', axis=1, inplace=True)

In [25]:
df.drop('listing_id', axis=1, inplace=True)

In [38]:
feature_df = pd.DataFrame(data=np.zeros((df.shape[0], len(all_features))), columns=all_features, dtype='uint8')

In [39]:
feature_df

,,expansive living,open house sit pm,private roof access,central park block away,valet,stainless,bicycle,bicycle storage,private garage available,...,roof deck stunning view,th walk,outdoor pool,security deposit,sleep alcove,elegant e s sprawl sundrenched home french drs breakfast bar dishwasher eat walk closet,health club pool,storage available,granite counter,wheelchair ramp
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34541,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34542,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34543,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34544,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [27]:
import catboost

X_train1, X_test1, y_train1, y_test1 = train_test_split(
    df.drop(columns=["display_address", "features", "latitude", "longitude", "street_address", "TARGET"]),
    df["TARGET"], train_size=0.8
)

In [44]:

params1 = {
    "iterations" : 1000,
    "depth" : 10,
}
X1 = df.drop(columns=["display_address", "features", "latitude", "longitude", "street_address", "TARGET"])
scaler = StandardScaler()
X1.loc[:, ["price", "bathrooms", "bedrooms"]] = scaler.fit_transform(X1.loc[:, ["price", "bathrooms", "bedrooms"]])
model1 = catboost.CatBoostClassifier(**params1)
model1.fit(X1, 
           df["TARGET"], 
           cat_features=["building_id", "manager_id"],
           text_features=["description"], 
           plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.094794
0:	learn: 1.0371025	total: 3.87s	remaining: 1h 4m 24s
1:	learn: 0.9880919	total: 7.28s	remaining: 1h 32s
2:	learn: 0.9466908	total: 10.6s	remaining: 58m 50s
3:	learn: 0.9130326	total: 14.1s	remaining: 58m 20s
4:	learn: 0.8860191	total: 17.5s	remaining: 57m 59s
5:	learn: 0.8625509	total: 22.3s	remaining: 1h 1m 39s
6:	learn: 0.8418549	total: 27.1s	remaining: 1h 4m
7:	learn: 0.8247010	total: 31.7s	remaining: 1h 5m 29s
8:	learn: 0.8079826	total: 36.3s	remaining: 1h 6m 41s
9:	learn: 0.7881304	total: 40.8s	remaining: 1h 7m 20s
10:	learn: 0.7699861	total: 45.3s	remaining: 1h 7m 56s
11:	learn: 0.7546748	total: 49.8s	remaining: 1h 8m 21s
12:	learn: 0.7412510	total: 54.3s	remaining: 1h 8m 40s
13:	learn: 0.7308265	total: 58.7s	remaining: 1h 8m 51s
14:	learn: 0.7185255	total: 1m 3s	remaining: 1h 9m
15:	learn: 0.7070018	total: 1m 7s	remaining: 1h 9m 4s
16:	learn: 0.6969140	total: 1m 11s	remaining: 1h 9m 7s
17:	learn: 0.6884807	total: 1m 15s	remaining: 1h 9m 3s
18:	lear

149:	learn: 0.5314187	total: 10m 6s	remaining: 57m 18s
150:	learn: 0.5308433	total: 10m 10s	remaining: 57m 13s
151:	learn: 0.5303962	total: 10m 14s	remaining: 57m 9s
152:	learn: 0.5299872	total: 10m 18s	remaining: 57m 4s
153:	learn: 0.5296215	total: 10m 22s	remaining: 57m
154:	learn: 0.5290706	total: 10m 26s	remaining: 56m 55s
155:	learn: 0.5284382	total: 10m 30s	remaining: 56m 51s
156:	learn: 0.5280906	total: 10m 34s	remaining: 56m 47s
157:	learn: 0.5275195	total: 10m 38s	remaining: 56m 44s
158:	learn: 0.5267282	total: 10m 42s	remaining: 56m 39s
159:	learn: 0.5259046	total: 10m 46s	remaining: 56m 36s
160:	learn: 0.5256164	total: 10m 50s	remaining: 56m 31s
161:	learn: 0.5251707	total: 10m 54s	remaining: 56m 27s
162:	learn: 0.5249514	total: 10m 58s	remaining: 56m 23s
163:	learn: 0.5245840	total: 11m 2s	remaining: 56m 19s
164:	learn: 0.5239575	total: 11m 7s	remaining: 56m 15s
165:	learn: 0.5235632	total: 11m 11s	remaining: 56m 11s
166:	learn: 0.5232295	total: 11m 15s	remaining: 56m 7s
16

297:	learn: 0.4636335	total: 20m 18s	remaining: 47m 49s
298:	learn: 0.4630877	total: 20m 22s	remaining: 47m 45s
299:	learn: 0.4627438	total: 20m 26s	remaining: 47m 40s
300:	learn: 0.4623399	total: 20m 30s	remaining: 47m 36s
301:	learn: 0.4618722	total: 20m 33s	remaining: 47m 32s
302:	learn: 0.4615751	total: 20m 37s	remaining: 47m 27s
303:	learn: 0.4612997	total: 20m 41s	remaining: 47m 23s
304:	learn: 0.4609770	total: 20m 46s	remaining: 47m 19s
305:	learn: 0.4607063	total: 20m 50s	remaining: 47m 15s
306:	learn: 0.4602642	total: 20m 54s	remaining: 47m 10s
307:	learn: 0.4597735	total: 20m 58s	remaining: 47m 6s
308:	learn: 0.4592351	total: 21m 2s	remaining: 47m 2s
309:	learn: 0.4589619	total: 21m 6s	remaining: 46m 57s
310:	learn: 0.4583642	total: 21m 10s	remaining: 46m 53s
311:	learn: 0.4579340	total: 21m 14s	remaining: 46m 49s
312:	learn: 0.4575913	total: 21m 18s	remaining: 46m 45s
313:	learn: 0.4571489	total: 21m 22s	remaining: 46m 40s
314:	learn: 0.4567851	total: 21m 26s	remaining: 46m 

445:	learn: 0.4125816	total: 30m 8s	remaining: 37m 26s
446:	learn: 0.4123876	total: 30m 12s	remaining: 37m 21s
447:	learn: 0.4119966	total: 30m 16s	remaining: 37m 17s
448:	learn: 0.4114645	total: 30m 20s	remaining: 37m 13s
449:	learn: 0.4109694	total: 30m 24s	remaining: 37m 9s
450:	learn: 0.4107184	total: 30m 28s	remaining: 37m 5s
451:	learn: 0.4103925	total: 30m 32s	remaining: 37m 1s
452:	learn: 0.4099380	total: 30m 36s	remaining: 36m 57s
453:	learn: 0.4094232	total: 30m 40s	remaining: 36m 53s
454:	learn: 0.4091366	total: 30m 44s	remaining: 36m 48s
455:	learn: 0.4088554	total: 30m 48s	remaining: 36m 44s
456:	learn: 0.4086545	total: 30m 52s	remaining: 36m 40s
457:	learn: 0.4084201	total: 30m 56s	remaining: 36m 36s
458:	learn: 0.4080484	total: 31m	remaining: 36m 32s
459:	learn: 0.4078722	total: 31m 4s	remaining: 36m 28s
460:	learn: 0.4076454	total: 31m 8s	remaining: 36m 24s
461:	learn: 0.4074671	total: 31m 12s	remaining: 36m 20s
462:	learn: 0.4071887	total: 31m 16s	remaining: 36m 16s
46

593:	learn: 0.3716244	total: 39m 57s	remaining: 27m 18s
594:	learn: 0.3714089	total: 40m 1s	remaining: 27m 14s
595:	learn: 0.3711861	total: 40m 5s	remaining: 27m 10s
596:	learn: 0.3710126	total: 40m 9s	remaining: 27m 6s
597:	learn: 0.3707303	total: 40m 13s	remaining: 27m 2s
598:	learn: 0.3705218	total: 40m 17s	remaining: 26m 58s
599:	learn: 0.3703440	total: 40m 21s	remaining: 26m 54s
600:	learn: 0.3700659	total: 40m 25s	remaining: 26m 50s
601:	learn: 0.3695291	total: 40m 29s	remaining: 26m 46s
602:	learn: 0.3693791	total: 40m 33s	remaining: 26m 42s
603:	learn: 0.3691422	total: 40m 37s	remaining: 26m 38s
604:	learn: 0.3687851	total: 40m 41s	remaining: 26m 33s
605:	learn: 0.3686461	total: 40m 45s	remaining: 26m 29s
606:	learn: 0.3685589	total: 40m 49s	remaining: 26m 25s
607:	learn: 0.3683828	total: 40m 53s	remaining: 26m 21s
608:	learn: 0.3681666	total: 40m 57s	remaining: 26m 17s
609:	learn: 0.3678426	total: 41m 1s	remaining: 26m 13s
610:	learn: 0.3676282	total: 41m 5s	remaining: 26m 9s


741:	learn: 0.3377969	total: 49m 45s	remaining: 17m 18s
742:	learn: 0.3376118	total: 49m 49s	remaining: 17m 14s
743:	learn: 0.3374729	total: 49m 53s	remaining: 17m 10s
744:	learn: 0.3373119	total: 49m 57s	remaining: 17m 6s
745:	learn: 0.3369834	total: 50m 1s	remaining: 17m 2s
746:	learn: 0.3368312	total: 50m 5s	remaining: 16m 57s
747:	learn: 0.3366054	total: 50m 9s	remaining: 16m 53s
748:	learn: 0.3364106	total: 50m 13s	remaining: 16m 49s
749:	learn: 0.3361757	total: 50m 17s	remaining: 16m 45s
750:	learn: 0.3358917	total: 50m 21s	remaining: 16m 41s
751:	learn: 0.3355830	total: 50m 25s	remaining: 16m 37s
752:	learn: 0.3354663	total: 50m 29s	remaining: 16m 33s
753:	learn: 0.3352129	total: 50m 33s	remaining: 16m 29s
754:	learn: 0.3349655	total: 50m 37s	remaining: 16m 25s
755:	learn: 0.3347468	total: 50m 41s	remaining: 16m 21s
756:	learn: 0.3345569	total: 50m 45s	remaining: 16m 17s
757:	learn: 0.3344178	total: 50m 49s	remaining: 16m 13s
758:	learn: 0.3343022	total: 50m 53s	remaining: 16m 9

890:	learn: 0.3098430	total: 59m 37s	remaining: 7m 17s
891:	learn: 0.3096741	total: 59m 41s	remaining: 7m 13s
892:	learn: 0.3094856	total: 59m 45s	remaining: 7m 9s
893:	learn: 0.3092551	total: 59m 49s	remaining: 7m 5s
894:	learn: 0.3091160	total: 59m 53s	remaining: 7m 1s
895:	learn: 0.3090401	total: 59m 57s	remaining: 6m 57s
896:	learn: 0.3088735	total: 1h 1s	remaining: 6m 53s
897:	learn: 0.3086578	total: 1h 5s	remaining: 6m 49s
898:	learn: 0.3082943	total: 1h 9s	remaining: 6m 45s
899:	learn: 0.3080077	total: 1h 13s	remaining: 6m 41s
900:	learn: 0.3078256	total: 1h 17s	remaining: 6m 37s
901:	learn: 0.3076133	total: 1h 21s	remaining: 6m 33s
902:	learn: 0.3074945	total: 1h 25s	remaining: 6m 29s
903:	learn: 0.3072321	total: 1h 29s	remaining: 6m 25s
904:	learn: 0.3071476	total: 1h 33s	remaining: 6m 21s
905:	learn: 0.3070248	total: 1h 37s	remaining: 6m 17s
906:	learn: 0.3068522	total: 1h 41s	remaining: 6m 13s
907:	learn: 0.3066295	total: 1h 45s	remaining: 6m 9s
908:	learn: 0.3063948	total: 

In [ ]:
y_pred1 = model1.predict(X_test1)

In [28]:
print("RMSE 1 {}".format(balanced_accuracy_score(y_pred1, y_test1)))

In [48]:
import folium
def show_circles_on_map(data, latitude_column, longitude_column, color):
    """
    The function draws map with circles on it.
    The center of the map is the mean of coordinates passed in data.
    
    data: DataFrame that contains columns latitude_column and longitude_column
    latitude_column: string, the name of column for latitude coordinates
    longitude_column: string, the name of column for longitude coordinates
    color: string, the color of circles to be drawn
    """

    location = (data[latitude_column].mean(), data[longitude_column].mean())
    m = folium.Map(location=location)

    for _, row in data.iterrows():
        folium.Circle(
            radius=80,
            location=(row[latitude_column], row[longitude_column]),
            color=color,
            fill_color=color,
            fill=True
        ).add_to(m)

    return m


In [ ]:
show_circles_on_map(df.sample(20000), "latitude", "longitude", "green")

Поделим карту на районы(можно погуглить примерное расположение районов)

In [49]:
# сделаем границы для каждого из 10 районов(условных)
districs_area = dict()

districs_area[1] = [(40.76115, -74.00199),
                    (40.74455, -73.96035),
                    (40.71045, -73.98071),
                    (40.70811, -74.00028),
                    (40.69952, -74.00594),
                    (40.70884, -74.03729)
                   ]

districs_area[2] = [(40.82094, -73.95772),
                    (40.81159, -73.93518),
                    (40.80588, -73.93466),
                    (40.79964, -73.929),
                    (40.74815, -73.96801),
                    (40.76115, -74.00199)
                   ]

districs_area[3] = [(40.87799, -73.92904),
                    (40.86986, -73.90095),
                    (40.82444, -73.93493),
                    (40.81899, -73.91646),
                    (40.80037, -73.92881),
                    (40.81738, -73.96954)
                   ]
districs_area[4] = [(40.79472, -73.91858),
                    (40.774, -73.884),
                    (40.74628, -73.89933),
                    (40.76259, -73.94438)
                   ]

districs_area[5] = [(40.75557, -73.95203), 
                    (40.74296, -73.91582),
                    (40.69247, -73.92635),
                    (40.70899, -73.97486)
                   ]

districs_area[6] = [(40.70677, -73.99305),
                    (40.68475, -73.90604),
                    (40.65473, -73.91222),
                    (40.67625, -74.0212)
                   ]
districs_area[7] = [(40.66566, -74.02623),
                    (40.6589, -73.95329),
                    (40.58249, -73.99627),
                    (40.62695, -74.08346)
                   ]

In [50]:
def show_on_map(latitude_column, longitude_column, color):
    """
    The function draws map with circles on it.
    The center of the map is the mean of coordinates passed in data.
    
    data: DataFrame that contains columns latitude_column and longitude_column
    latitude_column: string, the name of column for latitude coordinates
    longitude_column: string, the name of column for longitude coordinates
    color: string, the color of circles to be drawn
    """

    location = (latitude_column.mean(), longitude_column.mean())
    m = folium.Map(location=location)

    for i in range(latitude_column.shape[0]):
        folium.Circle(
            radius=80,
            location=(latitude_column[i], longitude_column[i]),
            color=color,
            fill_color=color,
            fill=True
        ).add_to(m)

    return m


In [33]:
lat, lon = [], []
for dist, coord in districs_area.items():
    for lati, long in coord:
        lat.append(lati)
        lon.append(long)
    
show_on_map(np.array(lat), np.array(lon), 'red')

In [53]:


areas = []
areas.append(mplPath.Path(districs_area[1]))
areas.append(mplPath.Path(districs_area[2]))
areas.append(mplPath.Path(districs_area[3]))
areas.append(mplPath.Path(districs_area[4]))
areas.append(mplPath.Path(districs_area[5]))
areas.append(mplPath.Path(districs_area[6]))
areas.append(mplPath.Path(districs_area[7]))

def contained_in_area(point):
    for i in range(len(areas)):
        if areas[i].contains_point(point):
            return i + 1
    return -1 # other


In [54]:
coordinates = []
for row in range(df['latitude'].shape[0]):
    coordinates.append((df.loc[row, 'latitude'], df.loc[row, 'longitude']))
df['coordinates'] = coordinates

In [55]:
df['area'] = df['coordinates'].apply(lambda x : contained_in_area(x))

In [58]:
df.drop(columns=['latitude', 'longitude', 'coordinates'], inplace=True)

In [59]:
df.sample(2)

,bathrooms,bedrooms,building_id,description,display_address,features,manager_id,price,street_address,TARGET,month,day,photo,area
29372,1.0,1,5db6b2959a5890431559c9530c7fd107,beautiful upper east bond real estate broker s...,E 77 Street,[],85860e99ade732afc385111568f8e602,2250,307 E 77 Street,low,5,26,1,2
4083,1.0,2,bee9bad998da3743d98f7f11ce8f749d,look no no broker fee massive bed gourmet bran...,williamsburg - NO FEE,"['Common Outdoor Space', 'Private Outdoor Spac...",699c325b818541f314b691b76f3238d7,3500,"241 Meeker Ave,",low,4,11,1,5


In [51]:
d = dict()
for row in df['features'].apply(lambda x : x[1:-1]):
    features = row.split(',')
    for feature in features:
        feature = re.sub(r"^\s", '', feature.lower())[1:-1]
        #feature_df[row,]
        if d.get(feature) is None:
            d[feature] = 0
        d[feature] += 1

df_count = pd.DataFrame({"feature" : d.keys(), "count" : d.values()})        

df_count[df_count['count'] > 10].sort_values('count', ascending=False)
features = df_count[df_count['count'] > 10]['feature'].values

In [60]:
feature_df = df['features']

In [56]:
def preprocess_features(feature_text):
    features = feature_text[1:-1].split(',')
    new_features = []
    for feature in features:
        feature = re.sub(r"^\s", '', feature.lower())[1:-1]
        new_features.append(feature)
    return ' '.join(new_features)

In [57]:
df['features'] = df['features'].apply(lambda x : preprocess_features(x))

In [53]:
for row in range(df['features'].shape[0]):
    features = df.loc[row, 'features'][1:-1].split(',')
    for feature in features:
        feature = re.sub(r"^\s", '', feature.lower())[1:-1]
        if feature in feature_df.columns:
            feature_df.loc[row, feature] = 1

In [54]:
df = df.join(feature_df)

In [69]:
df

,bathrooms,bedrooms,building_id,description,display_address,features,manager_id,price,street_address,TARGET,month,day,photo,area
0,1.0,3,0,fabulous midtown west perfect update hardwood ...,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,laundry in unit no fee elevator,4bdc3d8c1aaa90d997ce2cb77680679b,4495,W 50 & AVE 10,medium,5,19,1,2
1,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,renovate,55 River Drive South,dogs allowed cats allowed no fee,e5808a5e6cc13988fe596704428d38d5,2570,55 River Drive South,medium,6,16,1,-1
2,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,rare best deal market perfect studio prime wes...,W 77 Street,elevator hardwood floors,d69d4e111612dd12ef864031c1148543,1795,22 W 77 Street,low,5,4,1,2
3,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,newly renovate flex offer new standard luxury ...,John Street,swimming pool doorman elevator fitness center ...,e6472c7237327dd3903b3d6f6a94515a,3400,100 John Street,low,5,12,1,1
4,1.0,3,fee4d465932160318364d9d48d272879,low fee not come like unit laundry low fee uni...,West 16th Street,laundry in building laundry in unit dishwasher...,6fba9b3a8327c607b8b043716efee684,5695,321 West 16th Street,low,6,16,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34541,1.0,1,14fdc4b01ae44b025f6c4d28c9097e5f,newly renovate locate th avenue n q train stat...,29th St,no pets,e6472c7237327dd3903b3d6f6a94515a,1950,30-95 29th St,low,6,16,1,4
34542,1.0,0,9b6cf886379a2511f8c633c84028efe7,newly renovate bleached plank hardwood recess ...,E 1st St,pre-war laundry in unit dishwasher hardwood fl...,5599e962719af3ccc2976855c2d5893c,2600,39 E 1st St,medium,5,10,1,1
34543,2.0,2,0,luxury hi rise prime select unit lounge restau...,50th Avenue,,e6472c7237327dd3903b3d6f6a94515a,5110,2-01 50th Avenue,low,4,21,1,-1
34544,1.0,2,8754cae39f6e053974aa2337017eb3c1,coopercooper com list id west th street apt g ...,400 West 56th Street,laundry in unit dishwasher cats allowed dogs a...,6e5c10246156ae5bdcd9b487ca99d96a,3400,400 West 56th Street,medium,5,14,1,2


In [111]:
def preprocess_address(adress):
    #adress = set(adress.split())
    regex = re.compile('[%s]+' % re.escape(string.punctuation))
    adress = regex.sub(' ', adress)
    adress = re.sub(r'\s+', ' ', re.sub("\d+", '', adress))
    adress = re.sub('street', 'st', adress)
    adress = ' '.join(sorted(list(set(adress.split()))))
    return adress

In [113]:
df['address'] = df['display_address'] + " "  + df['street_address']
df['address'] = df['address'].apply(lambda x : preprocess_address(str(x).lower()))

In [64]:
df.drop(columns=['longitude', 'latitude', 'coordinates'], inplace=True)

In [115]:
df

,Id,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,TARGET,address
0,57094,1.0,3,0,2016-05-19 18:06:27,A FABULOUS 3BR IN MIDTOWN WEST! PERFECT APAR...,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,"['Laundry In Unit', 'No Fee', 'Elevator']",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10,medium,amazing ave fee how is midtown no steal this w...
1,33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,Renovated Kitchen and Bathroom!,55 River Drive South,"['Dogs Allowed', 'Cats Allowed', 'No Fee']",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South,medium,drive river south
2,60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,RARE AND BEST DEAL ON THE MARKET!!!! PERFECT S...,W 77 Street,"['Elevator', 'Hardwood Floors']",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street,low,st w
3,53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,Newly renovated flex 2 apartment offers the ne...,John Street,"['Swimming Pool', 'Doorman', 'Elevator', 'Fitn...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street,low,john st
4,592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,LOW FEE apartments do not come around like thi...,West 16th Street,"['Laundry in Building', 'Laundry in Unit', 'Di...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street,low,st th west
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34541,25582,1.0,1,14fdc4b01ae44b025f6c4d28c9097e5f,2016-06-16 02:12:57,Newly renovated bedroom apartment located off ...,29th St,['No pets'],40.7653,7166867,-73.9248,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7166867_625fde3...,1950,30-95 29th St,low,st th
34542,50013,1.0,0,9b6cf886379a2511f8c633c84028efe7,2016-05-10 03:17:32,All apartments are newly renovated featuring: ...,E 1st St,"['Pre-War', 'Laundry in Unit', 'Dishwasher', '...",40.7239,6989251,-73.9901,5599e962719af3ccc2976855c2d5893c,['https://photos.renthop.com/2/6989251_42a4884...,2600,39 E 1st St,medium,e st
34543,111475,2.0,2,0,2016-04-21 03:29:35,"<![CDATA[2 bedrooms, 5110, Astoria / Long Isla...",50th Avenue,[],40.7437,6905568,-73.9585,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/6905568_01602e3...,5110,2-01 50th Avenue,low,avenue th
34544,71184,1.0,2,8754cae39f6e053974aa2337017eb3c1,2016-05-14 02:27:47,CooperCooper.com :: Listing ID #10_0385; 400 W...,400 West 56th Street,"['Laundry in Unit', 'Dishwasher', 'Cats Allowe...",40.7673,7010187,-73.9866,6e5c10246156ae5bdcd9b487ca99d96a,['https://photos.renthop.com/2/7010187_ab1db5d...,3400,400 West 56th Street,medium,st th west


In [67]:
X2 = df.drop("TARGET", axis=1)
X2.loc[:, ["bathrooms", "bedrooms", "price"]] = scaler.fit_transform(X2.loc[:, ["bathrooms", "bedrooms", "price"]])
target2 = df["TARGET"]

In [68]:
params2 = {
    "iterations" : 1000,
    "depth" : 10,
}

model2 = catboost.CatBoostClassifier(**params2)
model2.fit(X2, 
           target2, 
           cat_features=["building_id", "manager_id"],
           text_features=["description", "features", "address"], 
           plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.094794
0:	learn: 1.0353608	total: 7.49s	remaining: 2h 4m 42s
1:	learn: 0.9825336	total: 15.1s	remaining: 2h 5m 22s
2:	learn: 0.9402055	total: 25s	remaining: 2h 18m 32s
3:	learn: 0.9087800	total: 35.6s	remaining: 2h 27m 52s
4:	learn: 0.8796597	total: 46s	remaining: 2h 32m 42s
5:	learn: 0.8570986	total: 56.1s	remaining: 2h 34m 53s
6:	learn: 0.8347946	total: 1m 6s	remaining: 2h 36m 31s
7:	learn: 0.8166489	total: 1m 16s	remaining: 2h 37m 8s
8:	learn: 0.8001790	total: 1m 25s	remaining: 2h 37m 31s
9:	learn: 0.7847820	total: 1m 35s	remaining: 2h 37m 26s
10:	learn: 0.7669525	total: 1m 44s	remaining: 2h 36m 59s
11:	learn: 0.7510250	total: 1m 53s	remaining: 2h 36m 23s
12:	learn: 0.7378026	total: 2m 3s	remaining: 2h 36m 3s
13:	learn: 0.7273407	total: 2m 12s	remaining: 2h 35m 34s
14:	learn: 0.7164286	total: 2m 21s	remaining: 2h 35m 15s
15:	learn: 0.7084031	total: 2m 31s	remaining: 2h 34m 55s
16:	learn: 0.6979258	total: 2m 40s	remaining: 2h 34m 42s
17:	learn: 0.6897472	total:

143:	learn: 0.5328579	total: 23m 33s	remaining: 2h 20m 3s
144:	learn: 0.5324078	total: 23m 43s	remaining: 2h 19m 50s
145:	learn: 0.5320531	total: 23m 52s	remaining: 2h 19m 38s
146:	learn: 0.5317887	total: 24m 1s	remaining: 2h 19m 25s
147:	learn: 0.5310887	total: 24m 11s	remaining: 2h 19m 13s
148:	learn: 0.5305078	total: 24m 20s	remaining: 2h 19m 1s
149:	learn: 0.5299789	total: 24m 29s	remaining: 2h 18m 48s
150:	learn: 0.5291283	total: 24m 39s	remaining: 2h 18m 37s
151:	learn: 0.5284191	total: 24m 48s	remaining: 2h 18m 24s
152:	learn: 0.5281100	total: 24m 57s	remaining: 2h 18m 12s
153:	learn: 0.5277134	total: 25m 7s	remaining: 2h 18m
154:	learn: 0.5269793	total: 25m 16s	remaining: 2h 17m 48s
155:	learn: 0.5266558	total: 25m 25s	remaining: 2h 17m 35s
156:	learn: 0.5260836	total: 25m 35s	remaining: 2h 17m 22s
157:	learn: 0.5254433	total: 25m 44s	remaining: 2h 17m 10s
158:	learn: 0.5247447	total: 25m 53s	remaining: 2h 16m 59s
159:	learn: 0.5245393	total: 26m 3s	remaining: 2h 16m 46s
160:	l

285:	learn: 0.4795136	total: 45m 39s	remaining: 1h 53m 58s
286:	learn: 0.4792156	total: 45m 48s	remaining: 1h 53m 48s
287:	learn: 0.4788054	total: 45m 58s	remaining: 1h 53m 38s
288:	learn: 0.4786456	total: 46m 7s	remaining: 1h 53m 28s
289:	learn: 0.4785025	total: 46m 16s	remaining: 1h 53m 18s
290:	learn: 0.4784092	total: 46m 26s	remaining: 1h 53m 8s
291:	learn: 0.4782451	total: 46m 35s	remaining: 1h 52m 57s
292:	learn: 0.4776692	total: 46m 44s	remaining: 1h 52m 48s
293:	learn: 0.4775230	total: 46m 54s	remaining: 1h 52m 37s
294:	learn: 0.4772613	total: 47m 3s	remaining: 1h 52m 27s
295:	learn: 0.4769302	total: 47m 12s	remaining: 1h 52m 17s
296:	learn: 0.4766590	total: 47m 22s	remaining: 1h 52m 7s
297:	learn: 0.4765312	total: 47m 31s	remaining: 1h 51m 56s
298:	learn: 0.4764451	total: 47m 40s	remaining: 1h 51m 46s
299:	learn: 0.4759524	total: 47m 50s	remaining: 1h 51m 36s
300:	learn: 0.4754984	total: 47m 59s	remaining: 1h 51m 26s
301:	learn: 0.4753518	total: 48m 8s	remaining: 1h 51m 16s
30

424:	learn: 0.4436068	total: 1h 7m 17s	remaining: 1h 31m 2s
425:	learn: 0.4433486	total: 1h 7m 27s	remaining: 1h 30m 53s
426:	learn: 0.4432169	total: 1h 7m 36s	remaining: 1h 30m 43s
427:	learn: 0.4429689	total: 1h 7m 45s	remaining: 1h 30m 33s
428:	learn: 0.4427743	total: 1h 7m 55s	remaining: 1h 30m 24s
429:	learn: 0.4424809	total: 1h 8m 4s	remaining: 1h 30m 14s
430:	learn: 0.4421902	total: 1h 8m 14s	remaining: 1h 30m 5s
431:	learn: 0.4421183	total: 1h 8m 23s	remaining: 1h 29m 55s
432:	learn: 0.4419111	total: 1h 8m 32s	remaining: 1h 29m 45s
433:	learn: 0.4414181	total: 1h 8m 42s	remaining: 1h 29m 36s
434:	learn: 0.4412286	total: 1h 8m 51s	remaining: 1h 29m 26s
435:	learn: 0.4408591	total: 1h 9m 1s	remaining: 1h 29m 16s
436:	learn: 0.4408028	total: 1h 9m 10s	remaining: 1h 29m 7s
437:	learn: 0.4405712	total: 1h 9m 19s	remaining: 1h 28m 57s
438:	learn: 0.4402203	total: 1h 9m 29s	remaining: 1h 28m 48s
439:	learn: 0.4401260	total: 1h 9m 39s	remaining: 1h 28m 38s
440:	learn: 0.4399754	total: 

558:	learn: 0.4173812	total: 1h 28m 9s	remaining: 1h 9m 32s
559:	learn: 0.4172688	total: 1h 28m 18s	remaining: 1h 9m 23s
560:	learn: 0.4170949	total: 1h 28m 27s	remaining: 1h 9m 13s
561:	learn: 0.4169821	total: 1h 28m 36s	remaining: 1h 9m 3s
562:	learn: 0.4169527	total: 1h 28m 46s	remaining: 1h 8m 54s
563:	learn: 0.4167122	total: 1h 28m 55s	remaining: 1h 8m 44s
564:	learn: 0.4164189	total: 1h 29m 5s	remaining: 1h 8m 35s
565:	learn: 0.4163226	total: 1h 29m 14s	remaining: 1h 8m 25s
566:	learn: 0.4161815	total: 1h 29m 23s	remaining: 1h 8m 16s
567:	learn: 0.4159558	total: 1h 29m 33s	remaining: 1h 8m 6s
568:	learn: 0.4159130	total: 1h 29m 42s	remaining: 1h 7m 56s
569:	learn: 0.4157132	total: 1h 29m 51s	remaining: 1h 7m 47s
570:	learn: 0.4154101	total: 1h 30m 1s	remaining: 1h 7m 37s
571:	learn: 0.4153035	total: 1h 30m 10s	remaining: 1h 7m 28s
572:	learn: 0.4152387	total: 1h 30m 19s	remaining: 1h 7m 18s
573:	learn: 0.4151524	total: 1h 30m 36s	remaining: 1h 7m 14s
574:	learn: 0.4144585	total: 

697:	learn: 0.3952987	total: 1h 51m 48s	remaining: 48m 22s
698:	learn: 0.3951757	total: 1h 51m 58s	remaining: 48m 13s
699:	learn: 0.3950429	total: 1h 52m 8s	remaining: 48m 3s
700:	learn: 0.3949450	total: 1h 52m 18s	remaining: 47m 54s
701:	learn: 0.3948597	total: 1h 52m 29s	remaining: 47m 44s
702:	learn: 0.3946595	total: 1h 52m 39s	remaining: 47m 35s
703:	learn: 0.3945389	total: 1h 52m 49s	remaining: 47m 26s
704:	learn: 0.3944989	total: 1h 52m 59s	remaining: 47m 16s
705:	learn: 0.3940188	total: 1h 53m 9s	remaining: 47m 7s
706:	learn: 0.3938557	total: 1h 53m 19s	remaining: 46m 58s
707:	learn: 0.3937775	total: 1h 53m 29s	remaining: 46m 48s
708:	learn: 0.3935853	total: 1h 53m 39s	remaining: 46m 39s
709:	learn: 0.3934813	total: 1h 53m 49s	remaining: 46m 29s
710:	learn: 0.3933765	total: 1h 54m	remaining: 46m 20s
711:	learn: 0.3932619	total: 1h 54m 10s	remaining: 46m 10s
712:	learn: 0.3929726	total: 1h 54m 20s	remaining: 46m 1s
713:	learn: 0.3927540	total: 1h 54m 30s	remaining: 45m 52s
714:	l

839:	learn: 0.3747528	total: 2h 15m 24s	remaining: 25m 47s
840:	learn: 0.3747070	total: 2h 15m 33s	remaining: 25m 37s
841:	learn: 0.3743860	total: 2h 15m 43s	remaining: 25m 28s
842:	learn: 0.3743199	total: 2h 15m 52s	remaining: 25m 18s
843:	learn: 0.3742878	total: 2h 16m 2s	remaining: 25m 8s
844:	learn: 0.3740002	total: 2h 16m 11s	remaining: 24m 58s
845:	learn: 0.3738678	total: 2h 16m 21s	remaining: 24m 49s
846:	learn: 0.3737684	total: 2h 16m 30s	remaining: 24m 39s
847:	learn: 0.3735961	total: 2h 16m 39s	remaining: 24m 29s
848:	learn: 0.3734905	total: 2h 16m 49s	remaining: 24m 20s
849:	learn: 0.3734580	total: 2h 16m 58s	remaining: 24m 10s
850:	learn: 0.3733700	total: 2h 17m 7s	remaining: 24m
851:	learn: 0.3731414	total: 2h 17m 17s	remaining: 23m 50s
852:	learn: 0.3730065	total: 2h 17m 26s	remaining: 23m 41s
853:	learn: 0.3729784	total: 2h 17m 36s	remaining: 23m 31s
854:	learn: 0.3729352	total: 2h 17m 46s	remaining: 23m 21s
855:	learn: 0.3728631	total: 2h 17m 55s	remaining: 23m 12s
856:

980:	learn: 0.3573753	total: 2h 37m 55s	remaining: 3m 3s
981:	learn: 0.3572849	total: 2h 38m 5s	remaining: 2m 53s
982:	learn: 0.3571617	total: 2h 38m 14s	remaining: 2m 44s
983:	learn: 0.3570916	total: 2h 38m 23s	remaining: 2m 34s
984:	learn: 0.3569431	total: 2h 38m 33s	remaining: 2m 24s
985:	learn: 0.3568268	total: 2h 38m 42s	remaining: 2m 15s
986:	learn: 0.3567168	total: 2h 38m 52s	remaining: 2m 5s
987:	learn: 0.3566848	total: 2h 39m 1s	remaining: 1m 55s
988:	learn: 0.3565793	total: 2h 39m 11s	remaining: 1m 46s
989:	learn: 0.3564919	total: 2h 39m 21s	remaining: 1m 36s
990:	learn: 0.3563181	total: 2h 39m 30s	remaining: 1m 26s
991:	learn: 0.3562649	total: 2h 39m 39s	remaining: 1m 17s
992:	learn: 0.3562274	total: 2h 39m 49s	remaining: 1m 7s
993:	learn: 0.3562082	total: 2h 39m 58s	remaining: 57.9s
994:	learn: 0.3559916	total: 2h 40m 8s	remaining: 48.3s
995:	learn: 0.3557706	total: 2h 40m 17s	remaining: 38.6s
996:	learn: 0.3557219	total: 2h 40m 27s	remaining: 29s
997:	learn: 0.3556894	tota

In [ ]:
y_pred2 = model2.predict(X_test2)
print("RMSE 1 {}".format(balanced_accuracy_score(y_pred2, y_test2)))

In [55]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [43]:
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import optuna
from functools import partial

In [114]:
def get_priority(x):
    if x == 'low':
        return 0
    elif x == 'medium':
        return 1
    else:
        return 2

In [91]:
df.loc[:, ['bathrooms', 'bedrooms', 'price']] = StandardScaler().fit_transform(df.loc[:, ['bathrooms', 'bedrooms', 'price']])
target = df['TARGET']#.apply(lambda x : get_priority(x))
X = df.drop(columns=["TARGET"])

In [180]:
X

,bathrooms,bedrooms,building_id,description,features,manager_id,price,month,day,photo,area,address
0,-0.425590,1.306672,0,fabulous midtown west perfect update hardwood ...,laundry in unit no fee elevator,4bdc3d8c1aaa90d997ce2cb77680679b,0.023043,5,19,1,2,10 50 amazing ave fee how is midtown no ste...
1,-0.425590,-0.484290,9225efdfb57a50bf3ec17ebab082f94a,renovate,dogs allowed cats allowed no fee,e5808a5e6cc13988fe596704428d38d5,-0.050133,6,16,1,-1,55 drive river south
2,-0.425590,-1.379770,320de7d3cc88e50a7fbbcfde1e825d21,rare best deal market perfect studio prime wes...,elevator hardwood floors,d69d4e111612dd12ef864031c1148543,-0.079594,5,4,1,2,22 77 st w
3,-0.425590,0.411191,ce6d18bf3238e668b2bf23f4110b7b67,newly renovate flex offer new standard luxury ...,swimming pool doorman elevator fitness center ...,e6472c7237327dd3903b3d6f6a94515a,-0.018582,5,12,1,1,100 john st
4,-0.425590,1.306672,fee4d465932160318364d9d48d272879,low fee not come like unit laundry low fee uni...,laundry in building laundry in unit dishwasher...,6fba9b3a8327c607b8b043716efee684,0.068660,6,16,1,1,16th 321 st west
...,...,...,...,...,...,...,...,...,...,...,...,...
34541,-0.425590,-0.484290,14fdc4b01ae44b025f6c4d28c9097e5f,newly renovate locate th avenue n q train stat...,no pets,e6472c7237327dd3903b3d6f6a94515a,-0.073702,6,16,1,4,29th 30 95 st
34542,-0.425590,-1.379770,9b6cf886379a2511f8c633c84028efe7,newly renovate bleached plank hardwood recess ...,pre-war laundry in unit dishwasher hardwood fl...,5599e962719af3ccc2976855c2d5893c,-0.048993,5,10,1,1,1st 39 e st
34543,1.589685,0.411191,0,luxury hi rise prime select unit lounge restau...,,e6472c7237327dd3903b3d6f6a94515a,0.046422,4,21,1,-1,2 01 50th avenue
34544,-0.425590,0.411191,8754cae39f6e053974aa2337017eb3c1,coopercooper com list id west th street apt g ...,laundry in unit dishwasher cats allowed dogs a...,6e5c10246156ae5bdcd9b487ca99d96a,-0.018582,5,14,1,2,400 56th st west


In [87]:
cat_params = {'loss_function' : "MultiClass", 
              "cat_features" : ['building_id', 'manager_id'],
              "text_features" : ["description", "features", "address"]
             }
model = catboost.CatBoostClassifier(**cat_params)

In [129]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X, target, train_size=0.8, random_state=42)

In [140]:
model.fit(X, target, plot=True)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.170275
0:	learn: 0.9898399	total: 7.24s	remaining: 1h 14s
1:	learn: 0.9173166	total: 15.9s	remaining: 1h 6m 9s
2:	learn: 0.8645217	total: 25.3s	remaining: 1h 9m 49s
3:	learn: 0.8267236	total: 30.4s	remaining: 1h 2m 49s
4:	learn: 0.8045795	total: 39.4s	remaining: 1h 5m
5:	learn: 0.7677675	total: 48.5s	remaining: 1h 6m 32s
6:	learn: 0.7410539	total: 57.3s	remaining: 1h 7m 18s
7:	learn: 0.7222519	total: 1m 5s	remaining: 1h 7m 35s
8:	learn: 0.7008077	total: 1m 14s	remaining: 1h 7m 51s
9:	learn: 0.6811199	total: 1m 23s	remaining: 1h 8m 6s
10:	learn: 0.6683781	total: 1m 31s	remaining: 1h 8m 8s
11:	learn: 0.6584964	total: 1m 40s	remaining: 1h 8m 14s
12:	learn: 0.6467391	total: 1m 49s	remaining: 1h 8m 27s
13:	learn: 0.6392445	total: 1m 58s	remaining: 1h 8m 28s
14:	learn: 0.6331613	total: 2m 6s	remaining: 1h 8m 25s
15:	learn: 0.6274622	total: 2m 15s	remaining: 1h 8m 28s
16:	learn: 0.6207668	total: 2m 24s	remaining: 1h 8m 27s
17:	learn: 0.6165160	total: 2m 33s	remaining: 1

148:	learn: 0.4858337	total: 21m 40s	remaining: 51m 3s
149:	learn: 0.4855827	total: 21m 48s	remaining: 50m 54s
150:	learn: 0.4848887	total: 21m 57s	remaining: 50m 44s
151:	learn: 0.4846462	total: 22m 5s	remaining: 50m 35s
152:	learn: 0.4842502	total: 22m 14s	remaining: 50m 26s
153:	learn: 0.4835845	total: 22m 22s	remaining: 50m 17s
154:	learn: 0.4832553	total: 22m 31s	remaining: 50m 8s
155:	learn: 0.4823870	total: 22m 40s	remaining: 49m 59s
156:	learn: 0.4814063	total: 22m 48s	remaining: 49m 50s
157:	learn: 0.4808196	total: 22m 57s	remaining: 49m 40s
158:	learn: 0.4803983	total: 23m 5s	remaining: 49m 31s
159:	learn: 0.4799657	total: 23m 14s	remaining: 49m 22s
160:	learn: 0.4791409	total: 23m 22s	remaining: 49m 13s
161:	learn: 0.4789329	total: 23m 31s	remaining: 49m 4s
162:	learn: 0.4784950	total: 23m 39s	remaining: 48m 55s
163:	learn: 0.4783869	total: 23m 48s	remaining: 48m 46s
164:	learn: 0.4776691	total: 23m 56s	remaining: 48m 37s
165:	learn: 0.4768848	total: 24m 5s	remaining: 48m 28

296:	learn: 0.4255676	total: 43m 40s	remaining: 29m 50s
297:	learn: 0.4254851	total: 43m 51s	remaining: 29m 43s
298:	learn: 0.4252921	total: 44m 1s	remaining: 29m 35s
299:	learn: 0.4251455	total: 44m 12s	remaining: 29m 28s
300:	learn: 0.4250765	total: 44m 24s	remaining: 29m 21s
301:	learn: 0.4248777	total: 44m 36s	remaining: 29m 14s
302:	learn: 0.4247180	total: 44m 49s	remaining: 29m 8s
303:	learn: 0.4239431	total: 45m 2s	remaining: 29m 2s
304:	learn: 0.4234843	total: 45m 15s	remaining: 28m 55s
305:	learn: 0.4230874	total: 45m 26s	remaining: 28m 48s
306:	learn: 0.4228225	total: 45m 39s	remaining: 28m 42s
307:	learn: 0.4226521	total: 45m 51s	remaining: 28m 35s
308:	learn: 0.4223768	total: 46m 3s	remaining: 28m 27s
309:	learn: 0.4221468	total: 46m 15s	remaining: 28m 20s
310:	learn: 0.4220108	total: 46m 26s	remaining: 28m 13s
311:	learn: 0.4219201	total: 46m 38s	remaining: 28m 6s
312:	learn: 0.4216341	total: 46m 49s	remaining: 27m 58s
313:	learn: 0.4214261	total: 47m 1s	remaining: 27m 51s

443:	learn: 0.3851395	total: 1h 7m 50s	remaining: 8m 33s
444:	learn: 0.3850817	total: 1h 7m 59s	remaining: 8m 24s
445:	learn: 0.3848475	total: 1h 8m 7s	remaining: 8m 14s
446:	learn: 0.3846959	total: 1h 8m 16s	remaining: 8m 5s
447:	learn: 0.3845968	total: 1h 8m 24s	remaining: 7m 56s
448:	learn: 0.3844847	total: 1h 8m 33s	remaining: 7m 47s
449:	learn: 0.3837083	total: 1h 8m 42s	remaining: 7m 38s
450:	learn: 0.3834637	total: 1h 8m 51s	remaining: 7m 28s
451:	learn: 0.3833107	total: 1h 9m	remaining: 7m 19s
452:	learn: 0.3832745	total: 1h 9m 9s	remaining: 7m 10s
453:	learn: 0.3830435	total: 1h 9m 17s	remaining: 7m 1s
454:	learn: 0.3828202	total: 1h 9m 27s	remaining: 6m 52s
455:	learn: 0.3826153	total: 1h 9m 36s	remaining: 6m 43s
456:	learn: 0.3824308	total: 1h 9m 46s	remaining: 6m 33s
457:	learn: 0.3821737	total: 1h 9m 55s	remaining: 6m 24s
458:	learn: 0.3820723	total: 1h 10m 3s	remaining: 6m 15s
459:	learn: 0.3818051	total: 1h 10m 13s	remaining: 6m 6s
460:	learn: 0.3817527	total: 1h 10m 22s

In [ ]:
y_pred3 = model.predict(X_test3)
print("Accuracy : {}".format(balanced_accuracy_score(y_test3, y_pred3)))

In [ ]:
def objective(trial):
    _iterations = trial.suggest_int("iterations", 10, 1000) 
    _learning_rate = trial.suggest_float("learning_rate", 0.001, 1.0) 
    _depth = trial.suggest_int("depth", 2, 16)
    new_estimator = catboost.CatBoostClassifier(iterations=_iterations, 
                                       learning_rate=_learning_rate,
                                       depth=_depth, **cat_params)
    scores = cross_val_score(new_estimator, 
                             X, 
                             target, 
                             cv=KFold(n_splits=4, shuffle=True, random_state=42),
                             scoring="balanced_accuracy")
    return scores.mean()

In [191]:
def tune_params():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, n_jobs=-1)
    best_params = study.best_params
    best_score = study.best_value
    print(f"Best score: {best_score}\n")
    return best_params

In [ ]:
best_params = tune_params() # результат 1 : 0.495 с {'iterations': 13, 'learning_rate': 0.9154574119017769, 'depth': 8}

[I 2022-01-05 00:24:13,984] A new study created in memory with name: no-name-bb57c8a7-f2fd-461b-8f68-f548156be876
Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 0.9773303	total: 971ms	remaining: 7.77s
1:	learn: 0.8977246	total: 2.42s	remaining: 8.48s
2:	learn: 0.8428218	total: 4.01s	remaining: 8.02s
0:	learn: 0.9125424	total: 4.23s	remaining: 42.3s
3:	learn: 0.8113681	total: 5.82s	remaining: 7.28s
4:	learn: 0.7747680	total: 7.45s	remaining: 5.96s
5:	learn: 0.7496978	total: 9.12s	remaining: 4.56s
1:	learn: 0.8209989	total: 11.1s	remaining: 49.7s
0:	learn: 0.8157584	total: 9.78s	remaining: 1m 27s
6:	learn: 0.7305646	total: 11.8s	remaining: 3.36s
7:	learn: 0.7128271	total: 13.7s	remaining: 1.71s
0:	learn: 0.9275367	total: 15.2s	remaining: 1m 15s
8:	learn: 0.6974330	total: 15.8s	remaining: 0us
1:	learn: 0.7109278	total: 20.2s	remaining: 1m 20s
3:	learn: 0.7076417	total: 27.8s	remaining: 48.7s
0:	learn: 0.9662798	total: 2.41s	remaining: 19.3s
2:	learn: 0.6632734	total: 31.1s	remaining: 1m 12s
1:	learn: 0.8925588	total: 5.52s	remaining: 19.3s
4:	learn: 0.6828640	total: 37s	remaining: 44.4s
1:	learn: 0.8358873	total: 36.8s	remaining: 1m 13s

[I 2022-01-05 00:26:38,866] Trial 0 finished with value: 0.36121508300535116 and parameters: {'iterations': 9, 'learning_rate': 0.22229105236226332, 'depth': 4}. Best is trial 0 with value: 0.36121508300535116.


3:	learn: 0.7340648	total: 41.3s	remaining: 1m 12s
2:	learn: 0.6616303	total: 41.5s	remaining: 1m 36s
0:	learn: 0.8275902	total: 5.91s	remaining: 59.1s
4:	learn: 0.6991651	total: 48.1s	remaining: 57.7s
0:	learn: 0.9038517	total: 24.1s	remaining: 2m
3:	learn: 0.6355587	total: 49.9s	remaining: 1m 14s
1:	learn: 0.7218450	total: 13.5s	remaining: 1m
5:	learn: 0.6774522	total: 57.4s	remaining: 47.8s
4:	learn: 0.6243291	total: 57.5s	remaining: 57.5s
2:	learn: 0.6759144	total: 20.5s	remaining: 54.7s
6:	learn: 0.6654183	total: 1m 5s	remaining: 37.4s
1:	learn: 0.8266808	total: 40.5s	remaining: 1m 20s
5:	learn: 0.6151666	total: 1m 5s	remaining: 43.5s
3:	learn: 0.6475316	total: 27.9s	remaining: 48.9s
7:	learn: 0.6510161	total: 1m 15s	remaining: 28.2s
4:	learn: 0.6332426	total: 34.5s	remaining: 41.4s
6:	learn: 0.6081965	total: 1m 13s	remaining: 31.3s
8:	learn: 0.6384892	total: 1m 23s	remaining: 18.6s
5:	learn: 0.6271820	total: 42.1s	remaining: 35.1s
7:	learn: 0.6041380	total: 1m 20s	remaining: 20.2

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

...

3:	learn: 0.6111929	total: 27m 11s	remaining: 1h 7m 59s
7:	learn: 0.5094799	total: 2h 6m 17s	remaining: 1h 34m 43s
4:	learn: 0.6037985	total: 30m 40s	remaining: 55m 13s
5:	learn: 0.5905191	total: 34m 4s	remaining: 45m 26s
3:	learn: 0.6405537	total: 1h 6m 2s	remaining: 1h 55m 34s
5:	learn: 0.5638880	total: 3h 8m 9s	remaining: 5h 13m 35s
2:	learn: 0.8265428	total: 1h 26m 59s	remaining: 2h 24m 58s
6:	learn: 0.5752918	total: 38m 9s	remaining: 38m 9s
7:	learn: 0.5587217	total: 41m 36s	remaining: 31m 12s
8:	learn: 0.5021268	total: 2h 21m 21s	remaining: 1h 18m 31s
8:	learn: 0.5525826	total: 45m 1s	remaining: 25m
7:	learn: 0.5239827	total: 3h 28m 26s	remaining: 2h 10m 16s
2:	learn: 0.6043977	total: 2h 37m 26s	remaining: 11h 22m 16s
9:	learn: 0.5366985	total: 48m 26s	remaining: 19m 22s
4:	learn: 0.6139767	total: 1h 20m 37s	remaining: 1h 36m 44s
10:	learn: 0.5329927	total: 52m 17s	remaining: 14m 15s
11:	learn: 0.5185841	total: 56m	remaining: 9m 20s
9:	learn: 0.4911119	total: 2h 35m 41s	remaining

[I 2022-01-05 09:40:39,302] Trial 20 finished with value: 0.48479366896965664 and parameters: {'iterations': 14, 'learning_rate': 0.9506729870003962, 'depth': 12}. Best is trial 12 with value: 0.49535537662482765.


3:	learn: 0.7809130	total: 1h 54m 22s	remaining: 1h 54m 22s
6:	learn: 0.5349195	total: 3h 36m 48s	remaining: 4h 38m 44s
0:	learn: 0.7834479	total: 6m 56s	remaining: 1h 44m
10:	learn: 0.4846620	total: 2h 50m 36s	remaining: 46m 31s
8:	learn: 0.5144408	total: 3h 54m 45s	remaining: 1h 44m 20s
1:	learn: 0.6916234	total: 12m 22s	remaining: 1h 26m 38s
6:	learn: 0.5576058	total: 1h 49m 7s	remaining: 1h 2m 21s
2:	learn: 0.6521660	total: 19m 36s	remaining: 1h 24m 57s
11:	learn: 0.4720741	total: 3h 7m 30s	remaining: 31m 15s
3:	learn: 0.6209265	total: 26m 11s	remaining: 1h 18m 33s
7:	learn: 0.5486261	total: 2h 4m 12s	remaining: 46m 34s
4:	learn: 0.7431884	total: 2h 23m 30s	remaining: 1h 26m 6s
4:	learn: 0.6020607	total: 31m 32s	remaining: 1h 9m 23s
7:	learn: 0.5182351	total: 4h 8m 45s	remaining: 4h 8m 45s
9:	learn: 0.5047321	total: 4h 21m 15s	remaining: 1h 18m 22s
5:	learn: 0.5859759	total: 37m 16s	remaining: 1h 2m 7s
0:	learn: 0.7721636	total: 1h 13m 10s	remaining: 18h 17m 43s
12:	learn: 0.462353

[I 2022-01-05 10:53:07,378] Trial 10 finished with value: 0.46753161679825916 and parameters: {'iterations': 11, 'learning_rate': 0.6597148453031108, 'depth': 14}. Best is trial 12 with value: 0.49535537662482765.


13:	learn: 0.5381849	total: 1h 14m 14s	remaining: 10m 36s
0:	learn: 0.7724718	total: 22m 12s	remaining: 4h 48m 42s
14:	learn: 0.5305001	total: 1h 18m	remaining: 5m 12s
0:	learn: 0.7816938	total: 6m 14s	remaining: 1h 33m 30s
15:	learn: 0.5222569	total: 1h 22m 1s	remaining: 0us
1:	learn: 0.6942010	total: 11m 32s	remaining: 1h 20m 47s
6:	learn: 0.6923983	total: 3h 19m 40s	remaining: 28m 31s
2:	learn: 0.6584004	total: 15m 21s	remaining: 1h 6m 33s
11:	learn: 0.4835765	total: 5h 13m 33s	remaining: 26m 7s
0:	learn: 0.7654175	total: 6m 23s	remaining: 1h 35m 46s
3:	learn: 0.6298242	total: 18m 58s	remaining: 56m 55s
9:	learn: 0.4891177	total: 5h 6m 39s	remaining: 3h 3m 59s
1:	learn: 0.6797924	total: 38m 42s	remaining: 3h 52m 12s
1:	learn: 0.6812313	total: 12m 20s	remaining: 1h 26m 26s
4:	learn: 0.6036998	total: 22m 38s	remaining: 49m 49s
1:	learn: 0.6724157	total: 2h 13m 3s	remaining: 15h 31m 24s
4:	learn: 0.5356395	total: 4h 31m 18s	remaining: 9h 56m 52s
5:	learn: 0.5863246	total: 26m 17s	remai

In [5]:
test_df = pd.read_csv('test.csv')
test_df.head()

,Id,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address
0,19477,2.0,3,87c4e08083ac83618dd9fdbf849331fe,2016-06-21 03:21:59,Rent listed is net effective after one month f...,E 77 St.,"['Pre-War', 'Laundry in Unit', 'Dishwasher', '...",40.7716,7189305,-73.9544,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7189305_f47ed5c...,4675,339 E 77 St.
1,29937,1.0,2,850c3d4412aebb3335273ed95f3e4bd4,2016-06-11 03:47:06,A FABULOUS 2BR IN THE UPPER EAST SIDE! PERFECT...,E 112 Street,[],40.7944,7140670,-73.9401,3e9f0fa34d67e5d61aae56776ecb8cee,['https://photos.renthop.com/2/7140670_6c5824f...,2100,252 E 112 Street
2,109148,1.0,2,84ddf917a091828ab9baedc0c926470e,2016-04-17 02:45:19,This is a great true 2 bedroom in a prime Midt...,Ninth Avenue,"['Dogs Allowed', 'Cats Allowed']",40.7650,6886239,-73.9882,10b1ae0a38d50b7ba0cee612d14af9eb,['https://photos.renthop.com/2/6886239_533bbee...,2800,783 Ninth Avenue
3,65728,2.0,3,315a2f868a195b076e5be73411110da2,2016-05-07 11:44:35,Brand new custom renovation in beautiful three...,East 12th street,"['Laundry In Unit', 'No Fee', 'Washer/Dryer in...",40.7288,6981146,-73.9808,e9920062e07ee893c10e38d0259665b0,['https://photos.renthop.com/2/6981146_f4a7400...,5500,186 Avenue B
4,33623,1.0,3,9b525bacc8d294728c35f27c04e22f46,2016-06-24 07:33:26,"Welcome Home! Located on a Prime UWS Block, Ba...",West 86th&Columbus,"['Private Outdoor Space', 'Elevator', 'Laundry...",40.7864,7210864,-73.9713,6d389fbe372d4d30b17733caa9370f95,['https://photos.renthop.com/2/7210864_e8528f4...,3995,47 West 86th street


In [30]:
test_df['month'] = pd.to_datetime(test_df['created']).apply(lambda x : x.month)
test_df['day'] = pd.to_datetime(test_df['created']).apply(lambda x : x.day)
test_df.drop('created', axis=1, inplace=True)

In [31]:
test_df['description'].fillna('', inplace=True)
test_df['description'] = test_df['description'].apply(lambda x : preprocess_text(x))

In [32]:
cv = CountVectorizer()
count = cv.fit_transform(test_df['description'])
count_values = count.toarray().sum(axis=0)
vocab = cv.vocabulary_
df_count = pd.DataFrame(sorted([(count_values[i], k) for k,i in vocab.items()], reverse=True)
            ).rename(columns={0: 'count', 1:'grams'})

words_to_delete = df_count.loc[df_count['count'] <= 50, 'grams'].values

In [33]:
test_df['description'] = test_df['description'].apply(lambda x : delete_words(x))

In [34]:
test_df['photo'] = test_df.photos.apply(lambda x : 1 if len(x) > 2 else 0)

In [69]:
coordinates = []
for row in range(test_df['latitude'].shape[0]):
    coordinates.append((test_df.loc[row, 'latitude'], test_df.loc[row, 'longitude']))
test_df['coordinates'] = coordinates

In [70]:
test_df['area'] = test_df['coordinates'].apply(lambda x : contained_in_area(x))

In [71]:
test_df['features'] = test_df['features'].apply(lambda x : preprocess_features(x))

In [72]:
test_df['address'] = test_df['display_address'] + " "  + test_df['street_address']
test_df['address'] = test_df['address'].apply(lambda x : preprocess_address(str(x).lower()))

In [73]:
test_df.drop(columns=["Id", "coordinates", 'latitude', 'longitude', 'street_address', 'display_address'], inplace=True)

In [74]:
test_df.drop('listing_id', axis=1, inplace=True)

KeyError: "['listing_id'] not found in axis"

In [75]:
test_df.drop('photos', axis=1, inplace=True)

KeyError: "['photos'] not found in axis"

In [77]:
test_df.loc[:, ["bathrooms", "bedrooms", "price"]] = scaler.fit_transform(test_df.loc[:, ["bathrooms", "bedrooms", "price"]])

In [79]:
y_pred52 = model2.predict(test_df)

In [150]:
def from_num_to_priority(num):
    if num == 0:
        return "low"
    elif num == 1:
        return "medium"
    else:
        return "high"

In [183]:
y_pred = pd.Series(y_pred[:, 0]).apply(lambda x : from_num_to_priority(x))

In [181]:
test_df.loc[:, ['bathrooms', 'bedrooms', 'price']] = StandardScaler().fit_transform(test_df.loc[:, ['bathrooms', 'bedrooms', 'price']])

In [168]:
y_pred.rename("TARGET")

0           low
1           low
2           low
3           low
4           low
          ...  
14801       low
14802    medium
14803       low
14804       low
14805       low
Name: TARGET, Length: 14806, dtype: object

In [35]:
Id = test_df['Id']

In [184]:
res1 = pd.DataFrame({"Id" : Id.values, "TARGET" : y_pred.values})

In [185]:
res1.to_csv("D:/python/Tinkoff/result1.csv", index=False)

In [36]:
df.drop(columns=["display_address", "features", "latitude", "longitude", "street_address", "TARGET"])

,bathrooms,bedrooms,building_id,description,manager_id,price,month,day,photo
0,1.0,3,0,fabulous midtown west perfect update hardwood ...,4bdc3d8c1aaa90d997ce2cb77680679b,4495,5,19,1
1,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,renovate,e5808a5e6cc13988fe596704428d38d5,2570,6,16,1
2,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,rare best deal market perfect studio prime wes...,d69d4e111612dd12ef864031c1148543,1795,5,4,1
3,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,newly renovate flex offer new standard luxury ...,e6472c7237327dd3903b3d6f6a94515a,3400,5,12,1
4,1.0,3,fee4d465932160318364d9d48d272879,low fee not come like unit laundry low fee uni...,6fba9b3a8327c607b8b043716efee684,5695,6,16,1
...,...,...,...,...,...,...,...,...,...
34541,1.0,1,14fdc4b01ae44b025f6c4d28c9097e5f,newly renovate locate th avenue n q train stat...,e6472c7237327dd3903b3d6f6a94515a,1950,6,16,1
34542,1.0,0,9b6cf886379a2511f8c633c84028efe7,newly renovate bleached plank hardwood recess ...,5599e962719af3ccc2976855c2d5893c,2600,5,10,1
34543,2.0,2,0,luxury postwar hi rise prime pantry select uni...,e6472c7237327dd3903b3d6f6a94515a,5110,4,21,1
34544,1.0,2,8754cae39f6e053974aa2337017eb3c1,coopercooper com list id west th street apt g ...,6e5c10246156ae5bdcd9b487ca99d96a,3400,5,14,1


In [45]:
Test1 = test_df.drop(columns=["display_address", 
                                                "features", 
                                                "latitude", 
                                                "longitude", 
                                                "street_address", 
                                                "Id"])
Test1.loc[:, ["price", "bedrooms", "bathrooms"]] = scaler.fit_transform(Test1.loc[:, ["price", "bedrooms", "bathrooms"]])
y_pred51 = model1.predict(Test1)

In [80]:
res52 = pd.DataFrame({"Id" : Id.values, "TARGET" : y_pred52[:, 0]})

In [81]:
res52.to_csv("D:/python/Tinkoff/result52.csv", index=False)

In [97]:
cat_params

{'loss_function': 'MultiClass',
 'cat_features': ['building_id', 'manager_id'],
 'text_features': ['description', 'features', 'address']}

In [98]:
#for feature in ["description", "features", "address", "area"]:
test_model = catboost.CatBoostClassifier(depth=10, n_estimators=1000, **{'loss_function': 'MultiClass',
 'cat_features': ['building_id', 'manager_id'],
 'text_features': ['features', 'address']})
X_train, X_test, y_train, y_test = train_test_split(X2.drop("description", axis=1),
                                                   target, train_size=0.8)
test_model.fit(X_train, y_train)
print("Without {} score = {}".format("description", balanced_accuracy_score(y_test, test_model.predict(X_test))))

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

Learning rate set to 0.093721
0:	learn: 1.0353534	total: 4.28s	remaining: 1h 11m 15s
1:	learn: 0.9854059	total: 8.53s	remaining: 1h 10m 58s
2:	learn: 0.9429671	total: 12.8s	remaining: 1h 10m 42s
3:	learn: 0.9091414	total: 16.9s	remaining: 1h 9m 56s
4:	learn: 0.8815907	total: 22.9s	remaining: 1h 15m 54s
5:	learn: 0.8588056	total: 28.9s	remaining: 1h 19m 52s
6:	learn: 0.8358668	total: 34.8s	remaining: 1h 22m 17s
7:	learn: 0.8195551	total: 40.5s	remaining: 1h 23m 44s
8:	learn: 0.8031630	total: 46.2s	remaining: 1h 24m 50s
9:	learn: 0.7905377	total: 51.9s	remaining: 1h 25m 34s
10:	learn: 0.7787707	total: 57.6s	remaining: 1h 26m 17s
11:	learn: 0.7669678	total: 1m 3s	remaining: 1h 26m 58s
12:	learn: 0.7560396	total: 1m 8s	remaining: 1h 27m 15s
13:	learn: 0.7429951	total: 1m 14s	remaining: 1h 27m 15s
14:	learn: 0.7315083	total: 1m 19s	remaining: 1h 27m 9s
15:	learn: 0.7219143	total: 1m 24s	remaining: 1h 27m 1s
16:	learn: 0.7096193	total: 1m 30s	remaining: 1h 26m 58s
17:	learn: 0.6977754	total:

144:	learn: 0.5367757	total: 12m 43s	remaining: 1h 15m
145:	learn: 0.5365410	total: 12m 48s	remaining: 1h 14m 54s
146:	learn: 0.5363357	total: 12m 53s	remaining: 1h 14m 47s
147:	learn: 0.5357327	total: 12m 58s	remaining: 1h 14m 40s
148:	learn: 0.5355371	total: 13m 3s	remaining: 1h 14m 34s
149:	learn: 0.5354540	total: 13m 8s	remaining: 1h 14m 27s
150:	learn: 0.5343339	total: 13m 13s	remaining: 1h 14m 21s
151:	learn: 0.5335834	total: 13m 18s	remaining: 1h 14m 14s
152:	learn: 0.5333986	total: 13m 23s	remaining: 1h 14m 8s
153:	learn: 0.5321424	total: 13m 28s	remaining: 1h 14m 2s
154:	learn: 0.5318820	total: 13m 33s	remaining: 1h 13m 55s
155:	learn: 0.5314084	total: 13m 38s	remaining: 1h 13m 49s
156:	learn: 0.5311758	total: 13m 43s	remaining: 1h 13m 43s
157:	learn: 0.5308910	total: 13m 48s	remaining: 1h 13m 36s
158:	learn: 0.5303689	total: 13m 53s	remaining: 1h 13m 30s
159:	learn: 0.5301772	total: 13m 58s	remaining: 1h 13m 23s
160:	learn: 0.5299827	total: 14m 3s	remaining: 1h 13m 17s
161:	l

286:	learn: 0.4977475	total: 24m 44s	remaining: 1h 1m 27s
287:	learn: 0.4975975	total: 24m 49s	remaining: 1h 1m 21s
288:	learn: 0.4975418	total: 24m 54s	remaining: 1h 1m 16s
289:	learn: 0.4972895	total: 24m 59s	remaining: 1h 1m 11s
290:	learn: 0.4972255	total: 25m 4s	remaining: 1h 1m 5s
291:	learn: 0.4971740	total: 25m 9s	remaining: 1h 1m
292:	learn: 0.4967078	total: 25m 14s	remaining: 1h 55s
293:	learn: 0.4966359	total: 25m 19s	remaining: 1h 49s
294:	learn: 0.4965994	total: 25m 24s	remaining: 1h 44s
295:	learn: 0.4964937	total: 25m 30s	remaining: 1h 39s
296:	learn: 0.4962023	total: 25m 35s	remaining: 1h 33s
297:	learn: 0.4961443	total: 25m 40s	remaining: 1h 28s
298:	learn: 0.4960571	total: 25m 45s	remaining: 1h 22s
299:	learn: 0.4959900	total: 25m 50s	remaining: 1h 17s
300:	learn: 0.4955995	total: 25m 55s	remaining: 1h 12s
301:	learn: 0.4951935	total: 26m	remaining: 1h 7s
302:	learn: 0.4948275	total: 26m 5s	remaining: 1h 2s
303:	learn: 0.4947140	total: 26m 11s	remaining: 59m 56s
304:	

434:	learn: 0.4682910	total: 37m 19s	remaining: 48m 28s
435:	learn: 0.4682092	total: 37m 24s	remaining: 48m 23s
436:	learn: 0.4678919	total: 37m 29s	remaining: 48m 17s
437:	learn: 0.4677031	total: 37m 34s	remaining: 48m 12s
438:	learn: 0.4674628	total: 37m 39s	remaining: 48m 7s
439:	learn: 0.4674157	total: 37m 44s	remaining: 48m 2s
440:	learn: 0.4670345	total: 37m 49s	remaining: 47m 57s
441:	learn: 0.4669041	total: 37m 55s	remaining: 47m 52s
442:	learn: 0.4665641	total: 38m	remaining: 47m 46s
443:	learn: 0.4664460	total: 38m 5s	remaining: 47m 41s
444:	learn: 0.4660668	total: 38m 10s	remaining: 47m 36s
445:	learn: 0.4659916	total: 38m 15s	remaining: 47m 31s
446:	learn: 0.4658317	total: 38m 20s	remaining: 47m 26s
447:	learn: 0.4657803	total: 38m 25s	remaining: 47m 20s
448:	learn: 0.4656752	total: 38m 30s	remaining: 47m 15s
449:	learn: 0.4656038	total: 38m 35s	remaining: 47m 10s
450:	learn: 0.4655001	total: 38m 40s	remaining: 47m 5s
451:	learn: 0.4651158	total: 38m 46s	remaining: 47m
452:

582:	learn: 0.4444709	total: 49m 53s	remaining: 35m 41s
583:	learn: 0.4444011	total: 49m 58s	remaining: 35m 36s
584:	learn: 0.4443452	total: 50m 3s	remaining: 35m 31s
585:	learn: 0.4442213	total: 50m 9s	remaining: 35m 25s
586:	learn: 0.4440576	total: 50m 14s	remaining: 35m 20s
587:	learn: 0.4438048	total: 50m 19s	remaining: 35m 15s
588:	learn: 0.4437336	total: 50m 24s	remaining: 35m 10s
589:	learn: 0.4436944	total: 50m 29s	remaining: 35m 5s
590:	learn: 0.4435752	total: 50m 34s	remaining: 35m
591:	learn: 0.4434951	total: 50m 39s	remaining: 34m 54s
592:	learn: 0.4433703	total: 50m 44s	remaining: 34m 49s
593:	learn: 0.4431489	total: 50m 50s	remaining: 34m 44s
594:	learn: 0.4430987	total: 50m 55s	remaining: 34m 39s
595:	learn: 0.4430009	total: 51m	remaining: 34m 34s
596:	learn: 0.4428733	total: 51m 5s	remaining: 34m 29s
597:	learn: 0.4427312	total: 51m 10s	remaining: 34m 24s
598:	learn: 0.4426604	total: 51m 15s	remaining: 34m 19s
599:	learn: 0.4422541	total: 51m 20s	remaining: 34m 13s
600:

730:	learn: 0.4247451	total: 1h 2m 26s	remaining: 22m 58s
731:	learn: 0.4246534	total: 1h 2m 31s	remaining: 22m 53s
732:	learn: 0.4244668	total: 1h 2m 36s	remaining: 22m 48s
733:	learn: 0.4243893	total: 1h 2m 41s	remaining: 22m 43s
734:	learn: 0.4242915	total: 1h 2m 46s	remaining: 22m 38s
735:	learn: 0.4242416	total: 1h 2m 52s	remaining: 22m 33s
736:	learn: 0.4241725	total: 1h 2m 57s	remaining: 22m 27s
737:	learn: 0.4241191	total: 1h 3m 2s	remaining: 22m 22s
738:	learn: 0.4239749	total: 1h 3m 7s	remaining: 22m 17s
739:	learn: 0.4239459	total: 1h 3m 12s	remaining: 22m 12s
740:	learn: 0.4238736	total: 1h 3m 17s	remaining: 22m 7s
741:	learn: 0.4238331	total: 1h 3m 22s	remaining: 22m 2s
742:	learn: 0.4237694	total: 1h 3m 27s	remaining: 21m 57s
743:	learn: 0.4236215	total: 1h 3m 32s	remaining: 21m 51s
744:	learn: 0.4235265	total: 1h 3m 37s	remaining: 21m 46s
745:	learn: 0.4232445	total: 1h 3m 42s	remaining: 21m 41s
746:	learn: 0.4231771	total: 1h 3m 48s	remaining: 21m 36s
747:	learn: 0.4229

872:	learn: 0.4084813	total: 1h 14m 28s	remaining: 10m 50s
873:	learn: 0.4084041	total: 1h 14m 33s	remaining: 10m 44s
874:	learn: 0.4083548	total: 1h 14m 38s	remaining: 10m 39s
875:	learn: 0.4082690	total: 1h 14m 43s	remaining: 10m 34s
876:	learn: 0.4081432	total: 1h 14m 48s	remaining: 10m 29s
877:	learn: 0.4080631	total: 1h 14m 53s	remaining: 10m 24s
878:	learn: 0.4079932	total: 1h 14m 59s	remaining: 10m 19s
879:	learn: 0.4079551	total: 1h 15m 4s	remaining: 10m 14s
880:	learn: 0.4079344	total: 1h 15m 9s	remaining: 10m 9s
881:	learn: 0.4078740	total: 1h 15m 14s	remaining: 10m 3s
882:	learn: 0.4078470	total: 1h 15m 19s	remaining: 9m 58s
883:	learn: 0.4077824	total: 1h 15m 24s	remaining: 9m 53s
884:	learn: 0.4077333	total: 1h 15m 29s	remaining: 9m 48s
885:	learn: 0.4077065	total: 1h 15m 34s	remaining: 9m 43s
886:	learn: 0.4076719	total: 1h 15m 40s	remaining: 9m 38s
887:	learn: 0.4075771	total: 1h 15m 45s	remaining: 9m 33s
888:	learn: 0.4074425	total: 1h 15m 50s	remaining: 9m 28s
889:	lea

In [99]:
test_model = catboost.CatBoostClassifier(depth=10, n_estimators=1000, **{'loss_function': 'MultiClass',
 'cat_features': ['building_id', 'manager_id'],
 'text_features': ['description', 'address']})
X_train, X_test, y_train, y_test = train_test_split(X2.drop("features", axis=1),
                                                   target, train_size=0.8)
test_model.fit(X_train, y_train)
print("Without {} score = {}".format("features", balanced_accuracy_score(y_test, test_model.predict(X_test))))

Learning rate set to 0.093721
0:	learn: 1.0397101	total: 8.6s	remaining: 2h 23m 7s
1:	learn: 0.9903007	total: 17s	remaining: 2h 21m 36s
2:	learn: 0.9507225	total: 25.2s	remaining: 2h 19m 46s
3:	learn: 0.9174070	total: 33.3s	remaining: 2h 18m 9s
4:	learn: 0.8900070	total: 41.2s	remaining: 2h 16m 36s
5:	learn: 0.8655904	total: 49.2s	remaining: 2h 15m 45s
6:	learn: 0.8443552	total: 56.9s	remaining: 2h 14m 33s
7:	learn: 0.8265239	total: 1m 4s	remaining: 2h 13m 27s
8:	learn: 0.8091264	total: 1m 12s	remaining: 2h 12m 16s
9:	learn: 0.7944151	total: 1m 19s	remaining: 2h 11m 28s
10:	learn: 0.7819063	total: 1m 27s	remaining: 2h 10m 32s
11:	learn: 0.7730273	total: 1m 34s	remaining: 2h 9m 44s
12:	learn: 0.7644328	total: 1m 41s	remaining: 2h 8m 39s
13:	learn: 0.7508912	total: 1m 48s	remaining: 2h 7m 54s
14:	learn: 0.7380476	total: 1m 56s	remaining: 2h 7m 13s
15:	learn: 0.7268201	total: 2m 3s	remaining: 2h 6m 41s
16:	learn: 0.7166260	total: 2m 10s	remaining: 2h 6m 13s
17:	learn: 0.7075459	total: 2m 

144:	learn: 0.5390671	total: 17m 42s	remaining: 1h 44m 27s
145:	learn: 0.5387211	total: 17m 50s	remaining: 1h 44m 19s
146:	learn: 0.5383956	total: 17m 57s	remaining: 1h 44m 12s
147:	learn: 0.5379138	total: 18m 4s	remaining: 1h 44m 4s
148:	learn: 0.5374672	total: 18m 12s	remaining: 1h 43m 57s
149:	learn: 0.5372073	total: 18m 19s	remaining: 1h 43m 50s
150:	learn: 0.5368778	total: 18m 26s	remaining: 1h 43m 42s
151:	learn: 0.5365127	total: 18m 33s	remaining: 1h 43m 34s
152:	learn: 0.5360961	total: 18m 41s	remaining: 1h 43m 27s
153:	learn: 0.5358931	total: 18m 48s	remaining: 1h 43m 18s
154:	learn: 0.5357218	total: 18m 55s	remaining: 1h 43m 11s
155:	learn: 0.5353519	total: 19m 2s	remaining: 1h 43m 2s
156:	learn: 0.5349819	total: 19m 9s	remaining: 1h 42m 54s
157:	learn: 0.5345476	total: 19m 17s	remaining: 1h 42m 46s
158:	learn: 0.5338825	total: 19m 24s	remaining: 1h 42m 39s
159:	learn: 0.5333278	total: 19m 31s	remaining: 1h 42m 31s
160:	learn: 0.5329345	total: 19m 39s	remaining: 1h 42m 24s
16

284:	learn: 0.4834724	total: 34m 45s	remaining: 1h 27m 11s
285:	learn: 0.4832449	total: 34m 52s	remaining: 1h 27m 4s
286:	learn: 0.4827936	total: 35m	remaining: 1h 26m 57s
287:	learn: 0.4823873	total: 35m 7s	remaining: 1h 26m 49s
288:	learn: 0.4818775	total: 35m 14s	remaining: 1h 26m 42s
289:	learn: 0.4815217	total: 35m 22s	remaining: 1h 26m 35s
290:	learn: 0.4812913	total: 35m 29s	remaining: 1h 26m 28s
291:	learn: 0.4810408	total: 35m 36s	remaining: 1h 26m 20s
292:	learn: 0.4807145	total: 35m 44s	remaining: 1h 26m 13s
293:	learn: 0.4805022	total: 35m 51s	remaining: 1h 26m 6s
294:	learn: 0.4804487	total: 35m 58s	remaining: 1h 25m 59s
295:	learn: 0.4803525	total: 36m 6s	remaining: 1h 25m 51s
296:	learn: 0.4800859	total: 36m 13s	remaining: 1h 25m 44s
297:	learn: 0.4796237	total: 36m 20s	remaining: 1h 25m 36s
298:	learn: 0.4793998	total: 36m 27s	remaining: 1h 25m 29s
299:	learn: 0.4790877	total: 36m 35s	remaining: 1h 25m 22s
300:	learn: 0.4789280	total: 36m 42s	remaining: 1h 25m 14s
301:	

425:	learn: 0.4453137	total: 51m 52s	remaining: 1h 9m 54s
426:	learn: 0.4452039	total: 52m	remaining: 1h 9m 46s
427:	learn: 0.4449773	total: 52m 7s	remaining: 1h 9m 39s
428:	learn: 0.4446336	total: 52m 14s	remaining: 1h 9m 32s
429:	learn: 0.4444071	total: 52m 21s	remaining: 1h 9m 24s
430:	learn: 0.4442084	total: 52m 29s	remaining: 1h 9m 17s
431:	learn: 0.4439060	total: 52m 36s	remaining: 1h 9m 10s
432:	learn: 0.4437193	total: 52m 43s	remaining: 1h 9m 2s
433:	learn: 0.4431821	total: 52m 50s	remaining: 1h 8m 55s
434:	learn: 0.4429681	total: 52m 58s	remaining: 1h 8m 47s
435:	learn: 0.4428375	total: 53m 5s	remaining: 1h 8m 40s
436:	learn: 0.4426756	total: 53m 12s	remaining: 1h 8m 33s
437:	learn: 0.4424965	total: 53m 19s	remaining: 1h 8m 25s
438:	learn: 0.4423554	total: 53m 27s	remaining: 1h 8m 18s
439:	learn: 0.4421909	total: 53m 34s	remaining: 1h 8m 10s
440:	learn: 0.4419542	total: 53m 41s	remaining: 1h 8m 3s
441:	learn: 0.4417607	total: 53m 48s	remaining: 1h 7m 56s
442:	learn: 0.4414268	

568:	learn: 0.4148890	total: 1h 9m 11s	remaining: 52m 24s
569:	learn: 0.4146285	total: 1h 9m 18s	remaining: 52m 17s
570:	learn: 0.4144028	total: 1h 9m 25s	remaining: 52m 9s
571:	learn: 0.4142110	total: 1h 9m 33s	remaining: 52m 2s
572:	learn: 0.4140226	total: 1h 9m 40s	remaining: 51m 55s
573:	learn: 0.4138608	total: 1h 9m 47s	remaining: 51m 47s
574:	learn: 0.4135849	total: 1h 9m 54s	remaining: 51m 40s
575:	learn: 0.4134646	total: 1h 10m 1s	remaining: 51m 33s
576:	learn: 0.4133073	total: 1h 10m 9s	remaining: 51m 25s
577:	learn: 0.4131260	total: 1h 10m 16s	remaining: 51m 18s
578:	learn: 0.4129214	total: 1h 10m 23s	remaining: 51m 11s
579:	learn: 0.4128357	total: 1h 10m 30s	remaining: 51m 3s
580:	learn: 0.4124372	total: 1h 10m 38s	remaining: 50m 56s
581:	learn: 0.4123784	total: 1h 10m 45s	remaining: 50m 49s
582:	learn: 0.4122760	total: 1h 10m 52s	remaining: 50m 41s
583:	learn: 0.4118287	total: 1h 11m	remaining: 50m 34s
584:	learn: 0.4114803	total: 1h 11m 7s	remaining: 50m 27s
585:	learn: 0.

709:	learn: 0.3898945	total: 1h 26m 13s	remaining: 35m 13s
710:	learn: 0.3898231	total: 1h 26m 21s	remaining: 35m 5s
711:	learn: 0.3896891	total: 1h 26m 28s	remaining: 34m 58s
712:	learn: 0.3894952	total: 1h 26m 35s	remaining: 34m 51s
713:	learn: 0.3893482	total: 1h 26m 42s	remaining: 34m 44s
714:	learn: 0.3892192	total: 1h 26m 49s	remaining: 34m 36s
715:	learn: 0.3890631	total: 1h 26m 57s	remaining: 34m 29s
716:	learn: 0.3889183	total: 1h 27m 4s	remaining: 34m 22s
717:	learn: 0.3888375	total: 1h 27m 11s	remaining: 34m 14s
718:	learn: 0.3885865	total: 1h 27m 18s	remaining: 34m 7s
719:	learn: 0.3884541	total: 1h 27m 26s	remaining: 34m
720:	learn: 0.3883470	total: 1h 27m 33s	remaining: 33m 52s
721:	learn: 0.3881608	total: 1h 27m 40s	remaining: 33m 45s
722:	learn: 0.3879547	total: 1h 27m 47s	remaining: 33m 38s
723:	learn: 0.3878067	total: 1h 27m 55s	remaining: 33m 30s
724:	learn: 0.3874877	total: 1h 28m 2s	remaining: 33m 23s
725:	learn: 0.3873445	total: 1h 28m 9s	remaining: 33m 16s
726:	l

849:	learn: 0.3697637	total: 1h 43m 10s	remaining: 18m 12s
850:	learn: 0.3696410	total: 1h 43m 17s	remaining: 18m 5s
851:	learn: 0.3695694	total: 1h 43m 24s	remaining: 17m 57s
852:	learn: 0.3694416	total: 1h 43m 32s	remaining: 17m 50s
853:	learn: 0.3692845	total: 1h 43m 39s	remaining: 17m 43s
854:	learn: 0.3691843	total: 1h 43m 46s	remaining: 17m 35s
855:	learn: 0.3690658	total: 1h 43m 54s	remaining: 17m 28s
856:	learn: 0.3690133	total: 1h 44m 1s	remaining: 17m 21s
857:	learn: 0.3688790	total: 1h 44m 8s	remaining: 17m 14s
858:	learn: 0.3688092	total: 1h 44m 15s	remaining: 17m 6s
859:	learn: 0.3687107	total: 1h 44m 23s	remaining: 16m 59s
860:	learn: 0.3686391	total: 1h 44m 30s	remaining: 16m 52s
861:	learn: 0.3685842	total: 1h 44m 37s	remaining: 16m 44s
862:	learn: 0.3684715	total: 1h 44m 44s	remaining: 16m 37s
863:	learn: 0.3681550	total: 1h 44m 52s	remaining: 16m 30s
864:	learn: 0.3680930	total: 1h 44m 59s	remaining: 16m 23s
865:	learn: 0.3677190	total: 1h 45m 6s	remaining: 16m 15s
86

991:	learn: 0.3524616	total: 2h 22s	remaining: 58.2s
992:	learn: 0.3524415	total: 2h 29s	remaining: 51s
993:	learn: 0.3523425	total: 2h 37s	remaining: 43.7s
994:	learn: 0.3522601	total: 2h 44s	remaining: 36.4s
995:	learn: 0.3522009	total: 2h 51s	remaining: 29.1s
996:	learn: 0.3520778	total: 2h 58s	remaining: 21.8s
997:	learn: 0.3519041	total: 2h 1m 6s	remaining: 14.6s
998:	learn: 0.3517307	total: 2h 1m 13s	remaining: 7.28s
999:	learn: 0.3516678	total: 2h 1m 20s	remaining: 0us
Without features score = 0.5069609041718098


In [100]:
test_model = catboost.CatBoostClassifier(depth=10, n_estimators=1000, **{'loss_function': 'MultiClass',
 'cat_features': ['building_id', 'manager_id'],
 'text_features': ['features', 'description']})
X_train, X_test, y_train, y_test = train_test_split(X2.drop("address", axis=1),
                                                   target, train_size=0.8)
test_model.fit(X_train, y_train)
print("Without {} score = {}".format("address", balanced_accuracy_score(y_test, test_model.predict(X_test))))

Learning rate set to 0.093721
0:	learn: 1.0409590	total: 6.17s	remaining: 1h 42m 45s
1:	learn: 0.9902612	total: 12.5s	remaining: 1h 44m 18s
2:	learn: 0.9546485	total: 18.7s	remaining: 1h 43m 50s
3:	learn: 0.9208780	total: 25s	remaining: 1h 43m 36s
4:	learn: 0.8897817	total: 31s	remaining: 1h 42m 48s
5:	learn: 0.8611846	total: 37.1s	remaining: 1h 42m 28s
6:	learn: 0.8388069	total: 43.2s	remaining: 1h 42m 8s
7:	learn: 0.8195303	total: 49.1s	remaining: 1h 41m 32s
8:	learn: 0.8037031	total: 55.1s	remaining: 1h 41m 2s
9:	learn: 0.7877269	total: 1m	remaining: 1h 40m 30s
10:	learn: 0.7760745	total: 1m 6s	remaining: 1h 39m 47s
11:	learn: 0.7644143	total: 1m 12s	remaining: 1h 39m 14s
12:	learn: 0.7488683	total: 1m 18s	remaining: 1h 38m 42s
13:	learn: 0.7354195	total: 1m 23s	remaining: 1h 38m 7s
14:	learn: 0.7257760	total: 1m 29s	remaining: 1h 37m 28s
15:	learn: 0.7165411	total: 1m 34s	remaining: 1h 36m 48s
16:	learn: 0.7082247	total: 1m 39s	remaining: 1h 36m 10s
17:	learn: 0.6974205	total: 1m 4

144:	learn: 0.5192372	total: 13m 22s	remaining: 1h 18m 53s
145:	learn: 0.5188370	total: 13m 28s	remaining: 1h 18m 48s
146:	learn: 0.5184797	total: 13m 33s	remaining: 1h 18m 41s
147:	learn: 0.5178171	total: 13m 39s	remaining: 1h 18m 36s
148:	learn: 0.5173591	total: 13m 44s	remaining: 1h 18m 29s
149:	learn: 0.5168950	total: 13m 50s	remaining: 1h 18m 24s
150:	learn: 0.5163245	total: 13m 55s	remaining: 1h 18m 18s
151:	learn: 0.5155418	total: 14m 1s	remaining: 1h 18m 12s
152:	learn: 0.5147298	total: 14m 6s	remaining: 1h 18m 6s
153:	learn: 0.5138741	total: 14m 12s	remaining: 1h 18m
154:	learn: 0.5134199	total: 14m 17s	remaining: 1h 17m 55s
155:	learn: 0.5128644	total: 14m 22s	remaining: 1h 17m 48s
156:	learn: 0.5120245	total: 14m 28s	remaining: 1h 17m 42s
157:	learn: 0.5114538	total: 14m 33s	remaining: 1h 17m 36s
158:	learn: 0.5106325	total: 14m 39s	remaining: 1h 17m 31s
159:	learn: 0.5098188	total: 14m 44s	remaining: 1h 17m 26s
160:	learn: 0.5094703	total: 14m 50s	remaining: 1h 17m 19s
161:

285:	learn: 0.4468529	total: 26m 16s	remaining: 1h 5m 34s
286:	learn: 0.4464193	total: 26m 21s	remaining: 1h 5m 29s
287:	learn: 0.4459723	total: 26m 27s	remaining: 1h 5m 24s
288:	learn: 0.4454860	total: 26m 32s	remaining: 1h 5m 18s
289:	learn: 0.4453262	total: 26m 38s	remaining: 1h 5m 13s
290:	learn: 0.4447674	total: 26m 44s	remaining: 1h 5m 8s
291:	learn: 0.4441981	total: 26m 49s	remaining: 1h 5m 2s
292:	learn: 0.4437652	total: 26m 55s	remaining: 1h 4m 57s
293:	learn: 0.4434517	total: 27m	remaining: 1h 4m 52s
294:	learn: 0.4429290	total: 27m 6s	remaining: 1h 4m 46s
295:	learn: 0.4427208	total: 27m 11s	remaining: 1h 4m 41s
296:	learn: 0.4424125	total: 27m 17s	remaining: 1h 4m 35s
297:	learn: 0.4419382	total: 27m 22s	remaining: 1h 4m 30s
298:	learn: 0.4414214	total: 27m 28s	remaining: 1h 4m 24s
299:	learn: 0.4410888	total: 27m 33s	remaining: 1h 4m 19s
300:	learn: 0.4407778	total: 27m 39s	remaining: 1h 4m 13s
301:	learn: 0.4403217	total: 27m 44s	remaining: 1h 4m 7s
302:	learn: 0.4398753	

431:	learn: 0.3932021	total: 39m 39s	remaining: 52m 8s
432:	learn: 0.3930961	total: 39m 44s	remaining: 52m 2s
433:	learn: 0.3927457	total: 39m 50s	remaining: 51m 57s
434:	learn: 0.3926531	total: 39m 55s	remaining: 51m 51s
435:	learn: 0.3922210	total: 40m 1s	remaining: 51m 45s
436:	learn: 0.3921123	total: 40m 6s	remaining: 51m 40s
437:	learn: 0.3917397	total: 40m 12s	remaining: 51m 34s
438:	learn: 0.3912911	total: 40m 17s	remaining: 51m 29s
439:	learn: 0.3909848	total: 40m 23s	remaining: 51m 24s
440:	learn: 0.3907212	total: 40m 28s	remaining: 51m 18s
441:	learn: 0.3904156	total: 40m 34s	remaining: 51m 13s
442:	learn: 0.3901560	total: 40m 39s	remaining: 51m 7s
443:	learn: 0.3899442	total: 40m 45s	remaining: 51m 2s
444:	learn: 0.3897013	total: 40m 50s	remaining: 50m 56s
445:	learn: 0.3893876	total: 40m 56s	remaining: 50m 51s
446:	learn: 0.3892315	total: 41m 1s	remaining: 50m 45s
447:	learn: 0.3887590	total: 41m 7s	remaining: 50m 40s
448:	learn: 0.3886772	total: 41m 13s	remaining: 50m 34s


579:	learn: 0.3549566	total: 53m 11s	remaining: 38m 30s
580:	learn: 0.3546929	total: 53m 16s	remaining: 38m 25s
581:	learn: 0.3545047	total: 53m 21s	remaining: 38m 19s
582:	learn: 0.3542921	total: 53m 27s	remaining: 38m 14s
583:	learn: 0.3541511	total: 53m 32s	remaining: 38m 8s
584:	learn: 0.3538228	total: 53m 38s	remaining: 38m 3s
585:	learn: 0.3537098	total: 53m 43s	remaining: 37m 57s
586:	learn: 0.3534904	total: 53m 49s	remaining: 37m 51s
587:	learn: 0.3533865	total: 53m 54s	remaining: 37m 46s
588:	learn: 0.3533175	total: 53m 59s	remaining: 37m 40s
589:	learn: 0.3527480	total: 54m 5s	remaining: 37m 35s
590:	learn: 0.3523544	total: 54m 11s	remaining: 37m 29s
591:	learn: 0.3518579	total: 54m 16s	remaining: 37m 24s
592:	learn: 0.3514649	total: 54m 21s	remaining: 37m 18s
593:	learn: 0.3513265	total: 54m 27s	remaining: 37m 13s
594:	learn: 0.3509916	total: 54m 33s	remaining: 37m 7s
595:	learn: 0.3507049	total: 54m 38s	remaining: 37m 2s
596:	learn: 0.3506269	total: 54m 43s	remaining: 36m 5

725:	learn: 0.3229151	total: 1h 6m 28s	remaining: 25m 5s
726:	learn: 0.3226705	total: 1h 6m 33s	remaining: 24m 59s
727:	learn: 0.3225110	total: 1h 6m 38s	remaining: 24m 54s
728:	learn: 0.3220878	total: 1h 6m 44s	remaining: 24m 48s
729:	learn: 0.3220059	total: 1h 6m 49s	remaining: 24m 43s
730:	learn: 0.3219545	total: 1h 6m 55s	remaining: 24m 37s
731:	learn: 0.3218052	total: 1h 7m	remaining: 24m 32s
732:	learn: 0.3217429	total: 1h 7m 5s	remaining: 24m 26s
733:	learn: 0.3215612	total: 1h 7m 11s	remaining: 24m 20s
734:	learn: 0.3212333	total: 1h 7m 16s	remaining: 24m 15s
735:	learn: 0.3210379	total: 1h 7m 22s	remaining: 24m 10s
736:	learn: 0.3207200	total: 1h 7m 28s	remaining: 24m 4s
737:	learn: 0.3201836	total: 1h 7m 33s	remaining: 23m 59s
738:	learn: 0.3199183	total: 1h 7m 39s	remaining: 23m 53s
739:	learn: 0.3196087	total: 1h 7m 44s	remaining: 23m 48s
740:	learn: 0.3193354	total: 1h 7m 49s	remaining: 23m 42s
741:	learn: 0.3192900	total: 1h 7m 55s	remaining: 23m 37s
742:	learn: 0.3188442

866:	learn: 0.2944706	total: 1h 19m 20s	remaining: 12m 10s
867:	learn: 0.2943870	total: 1h 19m 25s	remaining: 12m 4s
868:	learn: 0.2941870	total: 1h 19m 31s	remaining: 11m 59s
869:	learn: 0.2941412	total: 1h 19m 36s	remaining: 11m 53s
870:	learn: 0.2938763	total: 1h 19m 42s	remaining: 11m 48s
871:	learn: 0.2937937	total: 1h 19m 47s	remaining: 11m 42s
872:	learn: 0.2936852	total: 1h 19m 53s	remaining: 11m 37s
873:	learn: 0.2934461	total: 1h 19m 58s	remaining: 11m 31s
874:	learn: 0.2933098	total: 1h 20m 4s	remaining: 11m 26s
875:	learn: 0.2931643	total: 1h 20m 9s	remaining: 11m 20s
876:	learn: 0.2930326	total: 1h 20m 15s	remaining: 11m 15s
877:	learn: 0.2928601	total: 1h 20m 20s	remaining: 11m 9s
878:	learn: 0.2927677	total: 1h 20m 25s	remaining: 11m 4s
879:	learn: 0.2924732	total: 1h 20m 31s	remaining: 10m 58s
880:	learn: 0.2922564	total: 1h 20m 37s	remaining: 10m 53s
881:	learn: 0.2919806	total: 1h 20m 42s	remaining: 10m 47s
882:	learn: 0.2918628	total: 1h 20m 47s	remaining: 10m 42s
88

In [101]:
test_model = catboost.CatBoostClassifier(depth=10, n_estimators=1000, **{'loss_function': 'MultiClass',
 'cat_features': ['building_id', 'manager_id'],
 'text_features': ["description", 'features', 'address']})
X_train, X_test, y_train, y_test = train_test_split(X2.drop("area", axis=1),
                                                   target, train_size=0.8)
test_model.fit(X_train, y_train)
print("Without {} score = {}".format("area", balanced_accuracy_score(y_test, test_model.predict(X_test))))

Learning rate set to 0.093721
0:	learn: 1.0354879	total: 10.7s	remaining: 2h 57m 56s
1:	learn: 0.9869008	total: 21s	remaining: 2h 54m 34s
2:	learn: 0.9463665	total: 31s	remaining: 2h 51m 41s
3:	learn: 0.9113524	total: 40.9s	remaining: 2h 49m 32s
4:	learn: 0.8843773	total: 50.4s	remaining: 2h 47m 19s
5:	learn: 0.8606069	total: 60s	remaining: 2h 45m 34s
6:	learn: 0.8376957	total: 1m 9s	remaining: 2h 44m 40s
7:	learn: 0.8218151	total: 1m 18s	remaining: 2h 42m 47s
8:	learn: 0.8062344	total: 1m 27s	remaining: 2h 41m 28s
9:	learn: 0.7876268	total: 1m 37s	remaining: 2h 40m 3s
10:	learn: 0.7703340	total: 1m 46s	remaining: 2h 38m 50s
11:	learn: 0.7571217	total: 1m 54s	remaining: 2h 37m 40s
12:	learn: 0.7453175	total: 2m 3s	remaining: 2h 36m 38s
13:	learn: 0.7338144	total: 2m 12s	remaining: 2h 35m 36s
14:	learn: 0.7270635	total: 2m 21s	remaining: 2h 34m 36s
15:	learn: 0.7168858	total: 2m 30s	remaining: 2h 33m 56s
16:	learn: 0.7076514	total: 2m 38s	remaining: 2h 33m 12s
17:	learn: 0.6991090	total

143:	learn: 0.5385018	total: 21m 27s	remaining: 2h 7m 35s
144:	learn: 0.5376370	total: 21m 36s	remaining: 2h 7m 27s
145:	learn: 0.5370635	total: 21m 45s	remaining: 2h 7m 18s
146:	learn: 0.5367533	total: 21m 54s	remaining: 2h 7m 9s
147:	learn: 0.5364096	total: 22m 3s	remaining: 2h 6m 59s
148:	learn: 0.5362569	total: 22m 12s	remaining: 2h 6m 49s
149:	learn: 0.5358254	total: 22m 21s	remaining: 2h 6m 40s
150:	learn: 0.5351071	total: 22m 30s	remaining: 2h 6m 30s
151:	learn: 0.5345367	total: 22m 38s	remaining: 2h 6m 21s
152:	learn: 0.5343215	total: 22m 47s	remaining: 2h 6m 11s
153:	learn: 0.5336170	total: 22m 56s	remaining: 2h 6m 2s
154:	learn: 0.5328549	total: 23m 5s	remaining: 2h 5m 53s
155:	learn: 0.5323724	total: 23m 14s	remaining: 2h 5m 43s
156:	learn: 0.5320776	total: 23m 23s	remaining: 2h 5m 34s
157:	learn: 0.5316285	total: 23m 32s	remaining: 2h 5m 25s
158:	learn: 0.5311880	total: 23m 41s	remaining: 2h 5m 17s
159:	learn: 0.5308329	total: 23m 49s	remaining: 2h 5m 6s
160:	learn: 0.53037

284:	learn: 0.4860473	total: 42m 16s	remaining: 1h 46m 3s
285:	learn: 0.4858364	total: 42m 25s	remaining: 1h 45m 54s
286:	learn: 0.4856921	total: 42m 34s	remaining: 1h 45m 45s
287:	learn: 0.4852666	total: 42m 43s	remaining: 1h 45m 36s
288:	learn: 0.4850838	total: 42m 51s	remaining: 1h 45m 27s
289:	learn: 0.4849143	total: 43m	remaining: 1h 45m 18s
290:	learn: 0.4845389	total: 43m 9s	remaining: 1h 45m 9s
291:	learn: 0.4840964	total: 43m 18s	remaining: 1h 45m
292:	learn: 0.4838950	total: 43m 27s	remaining: 1h 44m 51s
293:	learn: 0.4837181	total: 43m 35s	remaining: 1h 44m 41s
294:	learn: 0.4835465	total: 43m 44s	remaining: 1h 44m 32s
295:	learn: 0.4834808	total: 43m 53s	remaining: 1h 44m 23s
296:	learn: 0.4832039	total: 44m 2s	remaining: 1h 44m 14s
297:	learn: 0.4828377	total: 44m 11s	remaining: 1h 44m 5s
298:	learn: 0.4823242	total: 44m 20s	remaining: 1h 43m 56s
299:	learn: 0.4822225	total: 44m 29s	remaining: 1h 43m 47s
300:	learn: 0.4821468	total: 44m 38s	remaining: 1h 43m 39s
301:	learn

424:	learn: 0.4487103	total: 1h 2m 54s	remaining: 1h 25m 6s
425:	learn: 0.4481660	total: 1h 3m 3s	remaining: 1h 24m 57s
426:	learn: 0.4479023	total: 1h 3m 12s	remaining: 1h 24m 48s
427:	learn: 0.4476369	total: 1h 3m 20s	remaining: 1h 24m 39s
428:	learn: 0.4475728	total: 1h 3m 29s	remaining: 1h 24m 30s
429:	learn: 0.4474043	total: 1h 3m 38s	remaining: 1h 24m 21s
430:	learn: 0.4471195	total: 1h 3m 47s	remaining: 1h 24m 12s
431:	learn: 0.4468907	total: 1h 3m 56s	remaining: 1h 24m 3s
432:	learn: 0.4467800	total: 1h 4m 5s	remaining: 1h 23m 54s
433:	learn: 0.4463782	total: 1h 4m 13s	remaining: 1h 23m 46s
434:	learn: 0.4462344	total: 1h 4m 22s	remaining: 1h 23m 37s
435:	learn: 0.4460748	total: 1h 4m 31s	remaining: 1h 23m 28s
436:	learn: 0.4456722	total: 1h 4m 40s	remaining: 1h 23m 19s
437:	learn: 0.4455479	total: 1h 4m 49s	remaining: 1h 23m 10s
438:	learn: 0.4452762	total: 1h 4m 58s	remaining: 1h 23m 1s
439:	learn: 0.4449314	total: 1h 5m 7s	remaining: 1h 22m 52s
440:	learn: 0.4446870	total: 1

559:	learn: 0.4173493	total: 1h 22m 55s	remaining: 1h 5m 9s
560:	learn: 0.4172977	total: 1h 23m 4s	remaining: 1h 5m
561:	learn: 0.4172084	total: 1h 23m 13s	remaining: 1h 4m 51s
562:	learn: 0.4171624	total: 1h 23m 22s	remaining: 1h 4m 42s
563:	learn: 0.4170738	total: 1h 23m 31s	remaining: 1h 4m 33s
564:	learn: 0.4169807	total: 1h 23m 39s	remaining: 1h 4m 24s
565:	learn: 0.4166793	total: 1h 23m 48s	remaining: 1h 4m 15s
566:	learn: 0.4166022	total: 1h 23m 57s	remaining: 1h 4m 6s
567:	learn: 0.4163303	total: 1h 24m 6s	remaining: 1h 3m 58s
568:	learn: 0.4160788	total: 1h 24m 15s	remaining: 1h 3m 49s
569:	learn: 0.4155835	total: 1h 24m 24s	remaining: 1h 3m 40s
570:	learn: 0.4154737	total: 1h 24m 33s	remaining: 1h 3m 31s
571:	learn: 0.4153530	total: 1h 24m 41s	remaining: 1h 3m 22s
572:	learn: 0.4150540	total: 1h 24m 50s	remaining: 1h 3m 13s
573:	learn: 0.4148493	total: 1h 24m 59s	remaining: 1h 3m 4s
574:	learn: 0.4144942	total: 1h 25m 8s	remaining: 1h 2m 55s
575:	learn: 0.4143980	total: 1h 25

699:	learn: 0.3910913	total: 1h 43m 36s	remaining: 44m 24s
700:	learn: 0.3907879	total: 1h 43m 44s	remaining: 44m 15s
701:	learn: 0.3906910	total: 1h 43m 53s	remaining: 44m 6s
702:	learn: 0.3905805	total: 1h 44m 2s	remaining: 43m 57s
703:	learn: 0.3904865	total: 1h 44m 11s	remaining: 43m 48s
704:	learn: 0.3904455	total: 1h 44m 20s	remaining: 43m 39s
705:	learn: 0.3903604	total: 1h 44m 29s	remaining: 43m 30s
706:	learn: 0.3900862	total: 1h 44m 38s	remaining: 43m 21s
707:	learn: 0.3899012	total: 1h 44m 47s	remaining: 43m 12s
708:	learn: 0.3896849	total: 1h 44m 56s	remaining: 43m 4s
709:	learn: 0.3894295	total: 1h 45m 4s	remaining: 42m 55s
710:	learn: 0.3892462	total: 1h 45m 13s	remaining: 42m 46s
711:	learn: 0.3891509	total: 1h 45m 22s	remaining: 42m 37s
712:	learn: 0.3888635	total: 1h 45m 31s	remaining: 42m 28s
713:	learn: 0.3884665	total: 1h 45m 40s	remaining: 42m 19s
714:	learn: 0.3883208	total: 1h 45m 49s	remaining: 42m 10s
715:	learn: 0.3882098	total: 1h 45m 57s	remaining: 42m 1s
71

840:	learn: 0.3712462	total: 2h 4m 26s	remaining: 23m 31s
841:	learn: 0.3709789	total: 2h 4m 35s	remaining: 23m 22s
842:	learn: 0.3708612	total: 2h 4m 44s	remaining: 23m 13s
843:	learn: 0.3707707	total: 2h 4m 53s	remaining: 23m 4s
844:	learn: 0.3705480	total: 2h 5m 1s	remaining: 22m 56s
845:	learn: 0.3704629	total: 2h 5m 10s	remaining: 22m 47s
846:	learn: 0.3703506	total: 2h 5m 19s	remaining: 22m 38s
847:	learn: 0.3702234	total: 2h 5m 28s	remaining: 22m 29s
848:	learn: 0.3700583	total: 2h 5m 37s	remaining: 22m 20s
849:	learn: 0.3696836	total: 2h 5m 45s	remaining: 22m 11s
850:	learn: 0.3695696	total: 2h 5m 54s	remaining: 22m 2s
851:	learn: 0.3694453	total: 2h 6m 3s	remaining: 21m 53s
852:	learn: 0.3692365	total: 2h 6m 12s	remaining: 21m 44s
853:	learn: 0.3691721	total: 2h 6m 21s	remaining: 21m 36s
854:	learn: 0.3690516	total: 2h 6m 29s	remaining: 21m 27s
855:	learn: 0.3689690	total: 2h 6m 38s	remaining: 21m 18s
856:	learn: 0.3688612	total: 2h 6m 47s	remaining: 21m 9s
857:	learn: 0.36881

982:	learn: 0.3522906	total: 2h 25m 25s	remaining: 2m 30s
983:	learn: 0.3521499	total: 2h 25m 34s	remaining: 2m 22s
984:	learn: 0.3519486	total: 2h 25m 43s	remaining: 2m 13s
985:	learn: 0.3518107	total: 2h 25m 51s	remaining: 2m 4s
986:	learn: 0.3517850	total: 2h 26m	remaining: 1m 55s
987:	learn: 0.3513115	total: 2h 26m 9s	remaining: 1m 46s
988:	learn: 0.3512044	total: 2h 26m 18s	remaining: 1m 37s
989:	learn: 0.3511461	total: 2h 26m 27s	remaining: 1m 28s
990:	learn: 0.3510027	total: 2h 26m 36s	remaining: 1m 19s
991:	learn: 0.3508362	total: 2h 26m 45s	remaining: 1m 11s
992:	learn: 0.3507622	total: 2h 26m 54s	remaining: 1m 2s
993:	learn: 0.3506549	total: 2h 27m 3s	remaining: 53.3s
994:	learn: 0.3505575	total: 2h 27m 12s	remaining: 44.4s
995:	learn: 0.3502716	total: 2h 27m 21s	remaining: 35.5s
996:	learn: 0.3501594	total: 2h 27m 30s	remaining: 26.6s
997:	learn: 0.3500325	total: 2h 27m 39s	remaining: 17.8s
998:	learn: 0.3500000	total: 2h 27m 48s	remaining: 8.88s
999:	learn: 0.3499218	total:

In [102]:
df

,bathrooms,bedrooms,building_id,description,features,manager_id,price,TARGET,month,day,photo,area,address
0,-0.425590,1.306672,0,fabulous midtown west perfect update hardwood ...,laundry in unit no fee elevator,4bdc3d8c1aaa90d997ce2cb77680679b,0.023043,medium,5,19,1,2,10 50 amazing ave fee how is midtown no ste...
1,-0.425590,-0.484290,9225efdfb57a50bf3ec17ebab082f94a,renovate,dogs allowed cats allowed no fee,e5808a5e6cc13988fe596704428d38d5,-0.050133,medium,6,16,1,-1,55 drive river south
2,-0.425590,-1.379770,320de7d3cc88e50a7fbbcfde1e825d21,rare best deal market perfect studio prime wes...,elevator hardwood floors,d69d4e111612dd12ef864031c1148543,-0.079594,low,5,4,1,2,22 77 st w
3,-0.425590,0.411191,ce6d18bf3238e668b2bf23f4110b7b67,newly renovate flex offer new standard luxury ...,swimming pool doorman elevator fitness center ...,e6472c7237327dd3903b3d6f6a94515a,-0.018582,low,5,12,1,1,100 john st
4,-0.425590,1.306672,fee4d465932160318364d9d48d272879,low fee not come like unit laundry low fee uni...,laundry in building laundry in unit dishwasher...,6fba9b3a8327c607b8b043716efee684,0.068660,low,6,16,1,1,16th 321 st west
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34541,-0.425590,-0.484290,14fdc4b01ae44b025f6c4d28c9097e5f,newly renovate locate th avenue n q train stat...,no pets,e6472c7237327dd3903b3d6f6a94515a,-0.073702,low,6,16,1,4,29th 30 95 st
34542,-0.425590,-1.379770,9b6cf886379a2511f8c633c84028efe7,newly renovate bleached plank hardwood recess ...,pre-war laundry in unit dishwasher hardwood fl...,5599e962719af3ccc2976855c2d5893c,-0.048993,medium,5,10,1,1,1st 39 e st
34543,1.589685,0.411191,0,luxury postwar hi rise prime pantry select uni...,,e6472c7237327dd3903b3d6f6a94515a,0.046422,low,4,21,1,-1,2 01 50th avenue
34544,-0.425590,0.411191,8754cae39f6e053974aa2337017eb3c1,coopercooper com list id west th street apt g ...,laundry in unit dishwasher cats allowed dogs a...,6e5c10246156ae5bdcd9b487ca99d96a,-0.018582,medium,5,14,1,2,400 56th st west


In [104]:
df["address"].apply(lambda x : re.sub(r'\s+', ' ', re.sub("\d+", '', x)))

0         amazing ave fee how is midtown no steal this ...
1                                        drive river south
2                                                     st w
3                                                  john st
4                                               th st west
                               ...                        
34541                                                th st
34542                                              st e st
34543                                            th avenue
34544                                           th st west
34545                                                 e st
Name: address, Length: 34546, dtype: object